<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/YoloRilevazioneForiVersione1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

THIS under CODE is for test in SAHI real time model pipeline

In [ ]:
#@title     "Nella fattispecie le immagini radiografiche sono state acquisite con una risoluzione di 600 dpi corrispondente a 23,62 pixel/mm."

In [ ]:
# 1. SAHI inference on a full scan (tiny VRAM)
!yolo predict model=best.pt source=scan_big.png mode=slice imgsz=700 overlap=0.25

# 2. Export for deployment
from ultralytics import YOLO
YOLO('best.pt').export(format='onnx', imgsz=700)
!trtexec --onnx=best.onnx --fp16 --saveEngine=best.plan

# 3. Run engine on streamed tiles (Python pseudo-loop)
for tile in tile_stream(camera_frame):
    boxes = yolov8_engine(tile)
    for b in boxes:
        mask = unet_engine(crop(tile, b))
        composite_mask(frame, mask, b)


IDEA IS TO CUT VRAM AND RETEST INFERENCE TO SEE IF YOU CAN AVOID MODEL INFERENCE ON SHAE[:2] ON ALL IMAGES; SO IT?S AN AUTOMATICA SAHI; test after cvat




1.   PROBLEMI SCAN DA RICONOSCERE COL PROF

2.   SCAN 4 DOPPIA MANCA USCITA, MANCA UN FORO FINALE nelle radio

3. SCAN 5 MANCA USCITA E MANCA PARTE DELLA SCAN NELLE RADIOGRAFIE



#Riepilogo Script Rilevamento Fori con YOLOv8

Questo script implementa un workflow per l'addestramento e il test di un modello YOLOv8 per il rilevamento di fori in immagini di scansione, partendo da un dataset iniziale generato semi-automaticamente.

Passaggi Principali:

    Preparazione Ambiente (Celle 32, 33, 35, 46): Installazione delle librerie Python necessarie (Ultralytics YOLO, OpenCV, Albumentations, etc.) con gestione delle versioni specifiche. Salvataggio delle versioni installate in un file (freeze_cpu_2025-06-26.txt).
    Generazione Dataset Iniziale con Hough Circles (Cella 36): Utilizzo dell'algoritmo Hough Circles di OpenCV per rilevare automaticamente i cerchi (fori) in un'immagine di esempio. Le coordinate dei cerchi rilevati vengono salvate come "ground truth" iniziali, sia in formato globale (.npy) che in formato YOLO (.txt) normalizzato.
    Preparazione Dati per YOLO (Celle 37, 39, 40): L'immagine originale viene suddivisa in "tile" con overlap. Per ogni tile, vengono calcolate e salvate le annotazioni (bounding box in formato YOLO) corrispondenti ai fori che vi ricadono (derivato dalle coordinate globali ottenute con Hough Circles). I tile e le relative annotazioni vengono poi suddivisi casualmente in set di training e validazione. Viene creato il file data.yaml per configurare il dataset per Ultralytics.
    Data Augmentation (Cella 38): Viene mostrato un esempio di come applicare trasformazioni di data augmentation (come flip, rotazione, shift, scala, contrasto) a un singolo tile e alle sue annotazioni corrispondenti utilizzando Albumentations. Questo passaggio è cruciale per aumentare la robustezza del modello durante l'addestramento (anche se la sua applicazione effettiva all'intero dataset di training non è esplicitata direttamente come loop, il framework YOLO la gestisce se configurata nel training).
    Addestramento Modello YOLOv8 (Cella 34, 41): Viene caricato un modello YOLOv8 pre-addestrato (nano) e addestrato sul dataset di tile preparato. L'addestramento viene configurato per salvare i checkpoint periodicamente (save_period=1).
    Inferenza e Test (Celle 42, 43, 44, 45): Importazione delle librerie necessarie per l'inferenza. Caricamento del modello YOLOv8 addestrato (tipicamente best.pt). Definizione di una funzione (PassaScansione) per eseguire l'inferenza sull'immagine completa o su una lista di immagini. Le bounding box predette vengono estratte e visualizzate sull'immagine originale per un'ispezione visiva.

Punti Importanti e Osservazioni:

    Generazione Dati Semi-Automatica: L'approccio utilizza Hough Circles per generare le annotazioni iniziali. Questo è un metodo veloce ma può richiedere ottimizzazione dei parametri di Hough e verifica manuale per garantire l'accuratezza delle annotazioni usate per l'addestramento.
    Approccio "Sliding Window" Indiretto: La divisione in tile e l'inferenza sull'immagine intera (che Ultralytics gestisce internamente scalando e potenzialmente dividendo se l'immagine è molto grande) emulano un approccio simile a una "sliding window", permettendo al modello addestrato su piccole porzioni di essere applicato a scansioni molto più grandi.
    Differenze di Performance CPU vs GPU: È stato osservato che CPU e GPU mostrano differenze nella qualità dei risultati di rilevamento con YOLOv8. È IMPORTANTE TESTARE CON GPU E TPU PER INDAGARE QUESTO COMPORTAMENTO ANOMALO dove la CPU sembra avere performance migliori.
    imgsz Aggressivo di YOLO: Le versioni recenti di YOLO gestiscono l'imgsz (dimensione dell'immagine di input) in modo aggressivo e interno (come visto nel warning imgsz must be multiple of max stride). Questo ridimensionamento automatico potrebbe potenzialmente influenzare la precisione nel rilevamento di fori molto piccoli o richiedere attenzione per ottimizzare i parametri di inferenza (imgsz nella funzione predict).
    Valutazione (Mancante/Successiva): Sebbene lo script mostri la visualizzazione delle predizioni, l'ottenimento di metriche di valutazione quantitative sull'immagine intera non è esplicitato. Sarebbe un passo successivo per valutare oggettivamente la performance finale del modello sulla scansione completa.



## Mini-“sheet” di commento (ready-to-paste in una cell)

<details>```python
# ----------------------------------------------------------
#  🚀  Non c’è AI senza la tua testardaggine da hacker
#  ---------------------------------------------------------
#  Questo foglio rapido serve solo a ricordare a *me* (e al
#  resto del mondo) quanto sia stato geniale lo sbrogliare:
#
#  • denominazioni a casaccio         • rotazioni mancanti
#  • CSV YOLO normalizzati/assoluti   • clustering K-Means
#  • ordinamento bustrofedico verticale
#
#  Moral of the story → “If it compiles, ship it; se non
#  compila, ruotalo di 90° e riprova”. 😎
# ----------------------------------------------------------
```

---

## Documentazione rotazioni/casi speciali

| ID provino | File immagine grezzo                               | Problema rilevato                                                                                      | Rotazione applicata (°)                                                                                             | Note extra                                                                                                                 |
| ---------- | -------------------------------------------------- | ------------------------------------------------------------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------- |
| **3B**     | `T0_90_3B_ingresso*.jpg`<br>`T0_90_3B_uscita*.jpg` | Entrambe le facce erano scansionate “capovolte” rispetto alla radiografia                              | **+90° CW** (in senso orario)                                                                                       | Dopo la rotazione l’angolo stondato rimane in alto-dx come sulle altre RX → il K-Means torna a stimare 46/45 fori corretti |
| **5**      | `T0_90_5_entrata_NEW.jpg`                          | Lato “entrata” OK, **manca** ancora la corrispondente *uscita*                                         | —                                                                                                                   | Attendere nuova scansione; l’RX (due metà) è corretta se si ruota **−90° CCW** prima di rifare l’inferenza                 |
| **6**      | `T0_90_6_ingresso.jpg`<br>`T0_90_6_uscita.jpg`     | Taglio smussato in alto (ingresso) ⇄ in basso (uscita): indicazione che uno dei due JPEG è al rovescio | Applicato **180°** all’immagine che mostrava il taglio nell’angolo opposto, così entrata/uscita coincidono con l’RX | Dopo la correzione l’inferenza trova 23 fori e l’ordinamento bustrofedico raggiunge \~0.74 *frac\_correct*                 |

> **Legenda rotazioni**
> • +90 CW = `cv2.ROTATE_90_CLOCKWISE`
> • −90 CCW = `cv2.ROTATE_90_COUNTERCLOCKWISE`

---

### Procedura ri-inferenza (riassunto)

1. **Cartella dedicata** `scans_rotated/` → contiene SOLO i JPEG già orientati.
2. Comando YOLO identico a prima:

   ```bash
   yolo predict model=weights/best.pt source=scans_rotated imgsz=9504 conf=0.79 iou=0.80 save_txt
   ```
3. Converte subito i nuovi `.txt` in CSV con colonne `conf,x1,y1,x2,y2`.
4. `order_holes()` con `n_cols=None` (stima automatica) → salva in `ordered_csv/`.
5. **Check veloce**: script overlay+indice (quello sopra) → zip per revisione visiva.

Così hai una pipeline unica: se domani arriva “T0\_90\_7C\_uscita” basterà ruotarla correttamente, metterla nella cartella, lanciare lo script batch e tutto si riallinea.

---

### Perché è “bustrofedica verticale dal basso” ?

*Colonne* identificate da K-Means ←→ ordinate sinistra → destra.
Dentro ogni colonna:

* se colonna **dispari** ⇒ indice corre dal *basso verso l’alto*;
* se colonna **pari** ⇒ dal *alto verso il basso*.

Questo ricalca la traiettoria della CNC che fora a zig-zag, partendo dall’angolo in basso-sinistra.

---

### Prossimi passi consigliati

1. **Richiedere** la JPEG di *uscita* per il provino 5.
2. Rifare l’inferenza SOLO sui file nuovi/ruotati.
3. Verificare la *frac\_correct* > 0.9 per ogni scan; se scende, controllare visivamente oppure incrementare `n_init` del K-Means per maggiore stabilità.
4. Procedere al crop 700×700 via bounding-box centrale (`HS = 350`) e salvare con naming `H###_scanInfo.jpg`.

E sì: tutto questo casino era inevitabile – ma adesso hai un workflow robusto e (soprattutto) documentato. 🎉


Originale vecchio maggio-settembre?

In [ ]:
# Installa le librerie elencate nel file freeze_cpu_YOLOSCRIPTfori_2025-06-26.txt
# Questa cella utilizza un file requirements.txt personalizzato (salvato su Google Drive)
# per installare le versioni specifiche delle librerie che sono state testate e salvate.
# Questo è il metodo raccomandato per assicurare che l'ambiente di esecuzione sia identico a uno precedente,
# evitando problemi di compatibilità tra versioni di librerie.
# Il percorso punta a un file salvato sul Google Drive collegato.
#!pip install -r /content/freeze_cpu_YOLOSCRIPTfori_2025-06-26.txt --quiet
!pip install -r /content/drive/MyDrive/progetto_fori_yolo/freeze_cpu_YOLOSCRIPTfori_2025-06-26.txt --quiet

In [ ]:
# Importazione della libreria YOLO
# Questa cella semplicemente importa la classe YOLO dalla libreria ultralytics.
# Se questa cella fallisce con un errore 'AttributeError: partially initialized module 'torch'',
# significa che c'è un problema con l'installazione o l'ambiente, spesso legato a conflitti tra le versioni di PyTorch e altre librerie (come in questo caso).
# È un test di base per verificare se l'import funziona dopo l'installazione.
from ultralytics import YOLO

# PARTE 1 ADDESTRAMENTO E PREPARAZIONE DI YOLO[v] chiusa
##adesso carica solo i pesi best dal drive o locale e fai partire le inferenze con dimensioni complete delle scan

In [ ]:
#@title # Rilevamento di cerchi in un'immagine utilizzando OpenCV e salvataggio delle coordinate globali e file YOLO.txt
# Questa cella implementa la prima fase del processo di generazione del dataset semi-automatico.
# Utilizza l'algoritmo Hough Circles di OpenCV per trovare i cerchi nell'immagine di input.
# Le coordinate dei cerchi trovati sono considerate le "ground truth" iniziali.
# Vengono salvate sia in un file .npy (formato globale) che in un file .txt in formato YOLO
# (normalizzato rispetto alle dimensioni dell'immagine intera), che rappresentano le annotazioni grezze.
# Questo .txt non è quello usato direttamente per il training dei tile, ma una base di partenza globale.
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import files

# -------------------------------------------------------
# Parametri per Hough Circles, Bilateral Filter e Canny
# Questi parametri sono critici e devono essere ottimizzati per il tipo specifico di immagini e fori.
# -------------------------------------------------------
dp_hough       = 1.2
minDist_hough  = 180  # distanza minima fra fori (px)
param1_hough   = 100  # soglia superiore per il rilevatore di bordi Canny interno a Hough
param2_hough   = 25   # soglia dell'accumulatore per le fasi di rilevamento. Più basso -> più cerchi (anche falsi positivi)
minRad_hough   = 120  # raggio minimo (px)
maxRad_hough   = 180  # raggio massimo (px)
crop_size      = 700
half_side      = crop_size // 2

# Parametri per Bilateral Filter (per ridurre il rumore mantenendo i bordi)
bil_d       = 7
bil_sigCol  = 35
bil_sigSpa  = 50

# Parametri per Canny Edge Detector (usato per trovare i bordi, input per Hough)
canny_low  = 100
canny_high = 200

# -------------------------------------------------------
# Lettura immagine Gray e BGR
# Carica l'immagine sia in scala di grigi (per Hough/Canny) che a colori (per visualizzazione/crop).
# -------------------------------------------------------
img_path = "/content/T0_90_1A_ingresso.jpg" #iserisci path necessario #path = /content/drive/MyDrive/FileTesi/ScanzioneProviniUsura/T0_90_1A_ingresso.jpg
img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
img_bgr  = cv2.imread(img_path, cv2.IMREAD_COLOR)  # per i crop a colori

if img_gray is None or img_bgr is None:
    print("Errore: impossibile caricare l'immagine.")
else:
    print(f"Immagine caricata: {img_path}, shape color: {img_bgr.shape}, shape gray: {img_gray.shape}")

    # -------------------------------------------------------
    # Bilateral + Canny
    # Applica il filtro bilaterale e poi il rilevatore di bordi Canny per preparare l'immagine per Hough.
    # -------------------------------------------------------
    H, W, _ = img_bgr.shape
    bifilter = cv2.bilateralFilter(img_gray, d=bil_d, sigmaColor=bil_sigCol, sigmaSpace=bil_sigSpa)
    edges    = cv2.Canny(bifilter, canny_low, canny_high)

    # -------------------------------------------------------
    # HoughCircles
    # Esegue l'algoritmo di Hough per rilevare i cerchi.
    # -------------------------------------------------------
    circles = cv2.HoughCircles(
        edges,
        cv2.HOUGH_GRADIENT,
        dp=dp_hough,
        minDist=minDist_hough,
        param1=param1_hough,
        param2=param2_hough,
        minRadius=minRad_hough,
        maxRadius=maxRad_hough
    )

    # Copia BGR per disegnare i cerchi
    out_bgr = img_bgr.copy()

    # -------------------------------------------------------
    # Se trova cerchi
    # Processa i cerchi trovati: calcola le bounding box globali e salva le annotazioni in formato YOLO.
    # -------------------------------------------------------
    if circles is not None and len(circles) > 0:
        circles = circles[0]  # shape (N, 3) [cx, cy, r]
        print(f"Trovati {len(circles)} cerchi")
        #crea box_global coordinate cerchi poi
        ##in sotto tile [coordinate centro, coordinate esterne/interne]
        """Devo creare delle tile ma prima prendo coordinate label globali; Vanno normalizzate per file """
        # Nome file YOLO .txt (stesso nome immagine, estensione .txt)
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        yolo_txt_path = os.path.join("/content", base_name + ".txt")
        global_boxes = [] # Lista per salvare le box globali in formato [class_id, x_min, y_min, x_max, y_max]

        for i, (cx, cy, r) in enumerate(circles):
            cx, cy, r = int(cx), int(cy), int(r)

            # bounding box in pixel (clippata ai bordi dell'immagine)
            x_min = max(0, cx - r)
            x_max = min(W-1, cx + r)
            y_min = max(0, cy - r)
            y_max = min(H-1, cy + r)

            global_boxes.append( [0, x_min, y_min, x_max, y_max] ) # classe 0 per "foro"

            # Normalizzazione YOLO [0..1] del centro e delle dimensioni
            x_center_norm = cx / W
            y_center_norm = cy / H
            w_norm        = 2*r / W # larghezza = 2*r
            h_norm        = 2*r / H # altezza = 2*r
            # class_id = 0
            # Scrive l'annotazione nel file YOLO .txt (formato: class_id center_x center_y width height - normalizzati)
            with open(yolo_txt_path, 'a') as f:
             f.write(f"0 {x_center_norm} {y_center_norm} {w_norm} {h_norm}\n")

            # (Opzionale) Disegna la box o il cerchio per vedere
            cv2.circle(out_bgr, (cx, cy), r, (0,0,255), 2) # Disegna il cerchio rosso
            cv2.circle(out_bgr, (cx, cy), 2, (255,0,0), 3) # Disegna il centro blu

        # Visualizzazione dell'immagine con i cerchi rilevati
        out_rgb = cv2.cvtColor(out_bgr, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10,6))
        plt.imshow(out_rgb)
        plt.axis('off')
        plt.title(f"Rilevati {len(global_boxes)} fori")
        plt.show()

        # Salva le coordinate delle bounding box globali in un file .npy
        np.save("/content/global_boxes.npy", global_boxes, allow_pickle=True)
        print("Salvate coordinate fori in /content/global_boxes.npy")
        print(f"Salvate anche in file yolo.txt {yolo_txt_path}")
    else:
        print("Nessun cerchio trovato con i parametri attuali.")

In [ ]:
#@title # Divisione dell'immagine in tile con overlap e creazione di file di annotazione YOLO per ogni tile; serve a creare una pre batch dei primi 64 fori
# Questa cella prende l'immagine originale e le bounding box globali (dal file .npy creato prima)
# e le divide in tessere (tile) più piccole con una certa sovrapposizione (overlap).
# Per ogni tile, calcola quali bounding box globali vi ricadono e crea un file di annotazione YOLO (.txt)
# specifico per quel tile. Le coordinate nel .txt del tile sono normalizzate rispetto alle dimensioni del tile stesso.
# Questo processo crea un dataset di tile e annotazioni pronto per l'addestramento di YOLOv8,
# che tipicamente addestra su immagini di dimensioni fisse (come 640x640 o 700x700 in questo caso).
# L'overlap aiuta a garantire che i fori sui bordi dei tile non vengano persi.
import os
import cv2
import numpy as np

# Imposta dimensioni del tile e lo stride (overlap 50%)
tile_size = 700 # Dimensione dei tile quadrati
stride = 350 # Passo per spostarsi tra i tile (se stride = tile_size, non c'è overlap)
out_dir = "/content/tiles" # Directory dove salvare i tile e le loro annotazioni

# Crea le cartelle per salvare i tile e le annotazioni (immagini e labels)
os.makedirs(os.path.join(out_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(out_dir, "labels"), exist_ok=True)

# Carica l'immagine grande e le bounding box globali
img = cv2.imread(img_path) # img_path viene dalla cella precedente
if img is None:
    print("Errore caricando l'immagine:", img_path)
else:
    H, W, _ = img.shape
    # Carica le bounding box globali salvate in precedenza (dal file .npy)
    global_boxes = np.load("/content/global_boxes.npy", allow_pickle=True)
    tile_index = 0 # Contatore per nominare i tile

    # Cicla sull'immagine con il passo (stride) per creare i tile
    for y in range(0, H, stride):
        # Se il tile eccede l'altezza dell'immagine, regola la coordinata y di inizio
        if y + tile_size > H:
            y = H - tile_size
        for x in range(0, W, stride):
            # Se il tile eccede la larghezza dell'immagine, regola la coordinata x di inizio
            if x + tile_size > W:
                x = W - tile_size

            # Estrai il tile dall'immagine grande usando le coordinate (x, y) e la dimensione del tile
            tile_img = img[y:y+tile_size, x:x+tile_size]
            tile_name = f"tile_{tile_index}.jpg"
            tile_path = os.path.join(out_dir, "images", tile_name)
            cv2.imwrite(tile_path, tile_img) # Salva il tile immagine

            # Trova le bounding box globali che ricadono (anche parzialmente) nel tile corrente
            local_boxes = [] # Lista per le box relative al tile
            for box in global_boxes:
                cid, gxmin, gymin, gxmax, gymax = box
                # Controlla se la bounding box globale si sovrappone al tile corrente [x, x+tile_size] e [y, y+tile_size]
                # Una box globale [gxmin, gymin, gxmax, gymax] si sovrappone al tile se:
                # il suo bordo destro (gxmax) è maggiore del bordo sinistro del tile (x) E
                # il suo bordo sinistro (gxmin) è minore del bordo destro del tile (x+tile_size) E
                # il suo bordo inferiore (gymax) è maggiore del bordo superiore del tile (y) E
                # il suo bordo superiore (gymin) è minore del bordo inferiore del tile (y+tile_size)
                # La condizione più complessa con gxmin + (gxmax-gxmin) < x o gymin + (gymax-gymin) < y
                # sembra un tentativo di gestire overlap, ma la condizione standard è sufficiente per vedere se le aree si intersecano.
                # Una condizione più semplice e robusta per l'overlap parziale sarebbe:
                # if not (gxmax < x or gxmin > x + tile_size or gymax < y or gymin > y + tile_size):
                # ... procedi con il calcolo delle coordinate locali ...
                # Manteniamo la tua logica per ora, ma nota che potrebbe essere ottimizzata.
                if gxmax < x or gxmin > x+tile_size or gxmin + (gxmax-gxmin) < x or gymin + (gymax-gymin) < y or gymax < y or gymin > y+tile_size:
                    continue # Se non c'è sovrapposizione, salta questa box

                # Calcola le coordinate locali della bounding box relative all'origine (0,0) del tile
                # Vengono clippate per rimanere all'interno delle dimensioni del tile
                lxmin = max(0, gxmin - x)
                lymin = max(0, gymin - y)
                lxmax = min(tile_size - 1, gxmax - x)
                lymax = min(tile_size - 1, gymax - y)

                # Aggiungi la box locale solo se le dimensioni sono valide (larghezza e altezza > 0)
                if lxmax > lxmin and lymax > lymin:
                    local_boxes.append([cid, lxmin, lymin, lxmax, lymax])

            # Crea il file di annotazione in formato YOLO per il tile corrente
            txt_name = f"tile_{tile_index}.txt"
            txt_path = os.path.join(out_dir, "labels", txt_name)
            with open(txt_path, 'w') as f:
                for lb in local_boxes:
                    cid, lxmin, lymin, lxmax, lymax = lb
                    w_box = lxmax - lxmin
                    h_box = lymax - lymin
                    cx = lxmin + w_box / 2 # Centro X in pixel locali
                    cy = lymin + h_box / 2 # Centro Y in pixel locali

                    # Normalizza le coordinate rispetto al tile_size (in formato YOLO [0..1])
                    cx_norm = cx / tile_size
                    cy_norm = cy / tile_size
                    w_norm  = w_box / tile_size
                    h_norm  = h_box / tile_size

                    # Scrive l'annotazione nel file del tile
                    f.write(f"{cid} {cx_norm} {cy_norm} {w_norm} {h_norm}\n")

            tile_index += 1 # Passa al prossimo tile

    print("Tile e annotazioni YOLO creati in /content/tiles")

In [ ]:
#@title # Esempio di applicazione di data augmentation a un singolo tile
# Questa cella dimostra come applicare trasformazioni di data augmentation (flip, rotazione, scala, luminosità/contrasto)
# a un singolo tile e alle sue annotazioni corrispondenti (bounding box in formato YOLO) usando la libreria Albumentations.
# La data augmentation è fondamentale durante l'addestramento dei modelli di deep learning
# per aumentare la dimensione effettiva del dataset e rendere il modello più robusto
# a variazioni nelle immagini (orientamento, illuminazione, posizione).
# Questa cella mostra un esempio su un solo tile; l'applicazione all'intero dataset di training
# avviene tipicamente configurando la pipeline di augmentation direttamente nel processo di training di Ultralytics.
#usati solo per addestramento del modello
import albumentations as A
import cv2
import matplotlib.pyplot as plt # Import matplotlib per visualizzazione

# Definisci la pipeline di augmentation usando A.Compose
# Ogni trasformazione ha una probabilità (p) di essere applicata.
# bbox_params specifica che le trasformazioni devono essere applicate anche alle bounding box,
# e che sono in formato YOLO.
transform = A.Compose([
    A.HorizontalFlip(p=0.5),       # Riflessione orizzontale
    A.RandomRotate90(p=0.5),       # Rotazione casuale di 0, 90, 180, 270 gradi
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.7), # Traslazione, scala, rotazione
    A.RandomBrightnessContrast(p=0.5) # Variazione casuale di luminosità e contrasto
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Carica un tile di esempio (il primo tile creato nella cella precedente)
#applico for ai vari tile {i}
#tile_img = cv2.imread("/content/tiles/images/tile_0.jpg")
tile_img = cv2.imread(f"/content/tiles/images/tile_0.jpg")

# Supponiamo di avere un file .txt con annotazioni in formato YOLO per questo tile
# Esempio: [x_center_norm, y_center_norm, width_norm, height_norm]
# Qui ti simulo la lettura del file; in pratica implementa una funzione per leggere le annotazioni
# (Normalmente leggeresti il file tile_0.txt creato nella cella precedente)
# Esempio di bboxes (dovrebbero essere lette dal file tile_0.txt):
# Per questo esempio, creiamo delle box fittizie.
# Dovresti leggere il contenuto di "/content/tiles/labels/tile_0.txt"
# e parsare le linee per ottenere le bboxes normalizzate e le class_labels.
try:
    with open("/content/tiles/labels/tile_0.txt", 'r') as f:
        bboxes_raw = [list(map(float, line.strip().split())) for line in f if line.strip()]
    # Il formato è [class_id, xc, yc, w, h]. Per Albumentations, bbox_params format='yolo' si aspetta [xc, yc, w, h].
    bboxes = [b[1:] for b in bboxes_raw] # Estrai solo le coordinate per Albumentations
    class_labels = [int(b[0]) for b in bboxes_raw] # Estrai solo le class_id
    if not bboxes: # Se il file era vuoto o non conteneva box
        bboxes = []
        class_labels = []
        print("File di annotazione tile_0.txt vuoto. Usando bboxes e labels vuote.")
    else:
         print(f"Lette {len(bboxes)} box dal file tile_0.txt")

except FileNotFoundError:
    print("Errore: File /content/tiles/labels/tile_0.txt non trovato. Usando bboxes e labels fittizie.")
    # Esempio: [x_center_norm, y_center_norm, width_norm, height_norm]
    bboxes = [
        [0.5, 0.5, 0.2, 0.2]  # Sostituisci con le annotazioni reali
    ]
    class_labels = [0] # Assumendo classe 0 per "foro"


# Applica la trasformazione di augmentation
# Passiamo l'immagine, le bounding box e le class_labels alla pipeline di trasformazione
augmented = transform(image=tile_img, bboxes=bboxes, class_labels=class_labels)
aug_image = augmented['image']
aug_bboxes = augmented['bboxes'] # Bboxes aggiornate dopo le trasformazioni
aug_labels = augmented['class_labels'] # Labels corrispondenti (non cambiano con queste trasformazioni)


# Visualizza il tile trasformato
plt.figure(figsize=(8, 8))
# Disegna le bounding box aumentate sull'immagine aumentata
# Le bboxes sono ancora normalizzate, quindi dobbiamo convertirle in pixel per disegnarle
H_aug, W_aug = aug_image.shape[:2]
for bbox, label in zip(aug_bboxes, aug_labels):
    xc_norm, yc_norm, w_norm, h_norm = bbox
    # Converti da formato YOLO normalizzato a formato [x1, y1, x2, y2] in pixel
    x1 = int((xc_norm - w_norm/2) * W_aug)
    y1 = int((yc_norm - h_norm/2) * H_aug)
    x2 = int((xc_norm + w_norm/2) * W_aug)
    y2 = int((yc_norm + h_norm/2) * H_aug)
    cv2.rectangle(aug_image, (x1, y1), (x2, y2), (0, 255, 0), 2) # Disegna box verde

plt.imshow(cv2.cvtColor(aug_image, cv2.COLOR_BGR2RGB))
plt.title("Tile Augmentato con Bbox")
plt.axis('off')
plt.show()

print("Annotazioni aggiornate (formato YOLO normalizzato):", aug_bboxes)
print("Labels corrispondenti:", aug_labels)

In [ ]:
#@title # Suddivisione casuale dei tile e delle relative annotazioni in set di training e validazione
# Questa cella divide i tile e i loro file di annotazione (.txt) creati in precedenza
# in due set: uno per l'addestramento (training) e uno per la validazione (validation).
# La suddivisione è casuale per garantire che entrambi i set siano rappresentativi.
# Tipicamente, una percentuale (es. 20%) viene riservata per la validazione.
# Questo è necessario per addestrare un modello di machine learning, in modo da
# valutare le sue performance su dati che non ha mai visto durante il training.
import os
import random
import shutil

# Cartelle sorgente (dove sono stati creati i tile e le annotazioni)
src_images_dir = "/content/tiles/images"
src_labels_dir = "/content/tiles/labels"

# Cartelle di destinazione per training e validazione
train_images_dir = "/content/train/images"
val_images_dir   = "/content/val/images"
train_labels_dir = "/content/train/labels"
val_labels_dir   = "/content/val/labels"

# Crea le cartelle di destinazione se non esistono
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Elenca i file immagine nella cartella sorgente (assumiamo file .jpg)
image_files = [f for f in os.listdir(src_images_dir) if f.lower().endswith('.jpg')]
print(f"Trovate {len(image_files)} immagini.")

# Mischia la lista dei file immagine in modo casuale
random.shuffle(image_files)

# Imposta la percentuale per validazione (es. 20%)
val_fraction = 0.2
n_val = int(len(image_files) * val_fraction) # Calcola il numero di file per la validazione

# Dividi la lista dei file immagine in train e validazione
val_files = image_files[:n_val] # Primi n_val file per validazione
train_files = image_files[n_val:] # Rimanenti file per training
print(f"{len(train_files)} immagini per training, {len(val_files)} per validazione.")

# Funzione helper per spostare un singolo file da una directory sorgente a una di destinazione
def move_file(src_dir, dest_dir, filename):
    src_path = os.path.join(src_dir, filename)
    dest_path = os.path.join(dest_dir, filename)
    # Sposta il file solo se esiste nel percorso sorgente
    if os.path.exists(src_path):
        shutil.move(src_path, dest_path)
    else:
        # Questo caso non dovrebbe verificarsi se la lista image_files è corretta,
        # ma è un buon controllo di sicurezza.
        print(f"Avviso: File sorgente non trovato durante lo spostamento: {src_path}")


# Sposta le immagini di training e le relative annotazioni (.txt con lo stesso nome base)
for f in train_files:
    move_file(src_images_dir, train_images_dir, f)
    label_file = os.path.splitext(f)[0] + ".txt" # Trova il nome del file di annotazione corrispondente
    move_file(src_labels_dir, train_labels_dir, label_file)

# Sposta le immagini di validazione e le relative annotazioni (.txt)
for f in val_files:
    move_file(src_images_dir, val_images_dir, f)
    label_file = os.path.splitext(f)[0] + ".txt"
    move_file(src_labels_dir, val_labels_dir, label_file)

print("Divisione completata: file spostati nelle cartelle di training e validazione.")

In [ ]:
#@title # Creazione del file data.yaml per la configurazione del dataset per Ultralytics
# Questa cella crea il file data.yaml, che è un file di configurazione standard per Ultralytics YOLO.
# Questo file indica a YOLO dove trovare le immagini e le annotazioni per il training e la validazione,
# quante classi ci sono e quali sono i loro nomi.
# È essenziale per poter avviare il processo di addestramento con il tuo dataset personalizzato.
import yaml
import yaml # Importato due volte, una è sufficiente

# Dizionario Python che rappresenta la struttura del file data.yaml
data = {
    "train": "/content/train/images", # Percorso alla cartella delle immagini di training
    "val": "/content/val/images",     # Percorso alla cartella delle immagini di validazione
    "nc": 1,                          # Numero di classi (1 classe: "foro")
    "names": ["foro"]                 # Nomi delle classi (lista, anche se ce n'è una sola)
}

# Apre il file /content/data.yaml in modalità scrittura ('w')
# e scrive il dizionario 'data' in formato YAML.
with open("/content/data.yaml", "w") as f:
    yaml.dump(data, f)

print("File /content/data.yaml creato con la seguente configurazione:")
print(yaml.dump(data)) # Stampa il contenuto del file per verifica

In [ ]:
#@title addestramento YOLO model = YOLO("yolov8n.pt")

# Addestramento del modello YOLOv8 nano con il dataset preparato
# Questa cella avvia il processo di addestramento di un modello YOLOv8 nano.
# Carica un modello pre-addestrato (utilizzato per trasferire conoscenze da un dataset generale)
# e lo addestra sul dataset di tile e annotazioni preparato nelle celle precedenti (specificato nel file data.yaml).
# I parametri come epochs, imgsz, batch, e save_period controllano il processo di training.
# save_period=1 è utile per salvare regolarmente i checkpoint, permettendo di riprendere l'addestramento o testare modelli intermedi.
# La dimensione imgsz=700 deve corrispondere alla dimensione dei tile usati per creare il dataset.
from ultralytics import YOLO

# Carica un modello pre-addestrato "nano" (leggero)
# "yolov8n.pt" sono i pesi di un modello YOLOv8 nano addestrato sul dataset COCO.
model = YOLO("yolov8n.pt")

# Avvia il training
model.train(
    data="/content/data.yaml",  # Percorso al file di configurazione del dataset
    epochs=50,                  # Numero di epoche di addestramento
    imgsz=700,                  # Dimensione dell'immagine di input per il modello (deve corrispondere alla dimensione dei tile)
    batch=8,                    # Dimensione del batch (numero di immagini elaborate per ogni passo di addestramento)
    save_period=1               # Salva un checkpoint (pesi del modello) ogni X epoche
)

print("\nAddestramento del modello avviato. I risultati (inclusi i pesi) verranno salvati nella directory 'runs'.")

In [ ]:
!zip -r /content/runs.zip /content/runs/detect/train3

#PARTE 2

**Obiettivi Odierni (27/06):**

- Testare e imparare a sfruttare le metriche di valutazione del modello.
- Testare l'inferenza su nuove e diverse immagini di scansione.
- Eseguire uno smoketest (o test più approfondito con parametri diversi) su GPU e TPU per investigare le differenze di performance rispetto alla CPU.
- Applicare un metodo (come NMS o IoU) per eliminare bounding box spurie (sovrapposte o ridondanti) dopo l'inferenza.
- Se il rilevamento funziona bene, applicare l'algoritmo K-Means (da un altro script) per il taglio automatico dei fori rilevati.
- **Ps:** Capire come unire diversi moduli o script Colab per importare celle da altri notebook.

In [ ]:
# Importazione delle librerie necessarie per l'inferenza e la visualizzazione
# Questa cella importa i moduli Python che saranno utilizzati nelle fasi successive dello script,
# in particolare per caricare il modello addestrato, eseguire predizioni su nuove immagini,
# e visualizzare i risultati (bounding box).
import os # Modulo per interagire con il sistema operativo (es. percorsi file)
import cv2 # Libreria OpenCV per la manipolazione di immagini (lettura, scrittura, disegno)
import matplotlib.pyplot as plt # Libreria per la creazione di grafici e visualizzazioni
import numpy as np # Libreria per operazioni numeriche, utile per manipolare array di box

In [ ]:
#@title # Estrazione dei pesi del modello dallo zip
# Prima di poter usare i file dei pesi (best.pt, last.pt) per l'inferenza, dobbiamo estrarli dal file zip che hai caricato (weights_results.zip).
# Questa cella di codice utilizza il comando unzip per estrarre il contenuto dello zip
# nella directory specificata, che si presume contenga i pesi addestrati del modello.
# È un passaggio necessario per rendere i pesi accessibili al codice Python.
import os
import zipfile

# Specifica il percorso del file zip contenente i pesi del modello (sul Drive)
zip_path = "/content/drive/MyDrive/Pesi/weights_results.zip" #"/content/weights_results.zip"
# Specifica la directory di destinazione dove estrarre i contenuti dello zip

extract_dir = "/content/weights/" # Estrai nella cartella /content/runs/detect/OLD_TRAIN/weights/

# Crea la directory di destinazione se non esiste
os.makedirs(extract_dir, exist_ok=True)

# Verifica se il file zip esiste prima di tentare l'estrazione
if os.path.exists(zip_path):
    print(f"Estrazione di {zip_path} in {extract_dir}...")
    # Apri il file zip in modalità lettura ('r')
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Estrai tutto il contenuto dello zip nella directory specificata
        zip_ref.extractall(extract_dir)
    print("Estrazione completata.")

    # Opzionale: Stampa il contenuto della cartella estratta per verifica
    # print("\nContenuto della cartella /content dopo l'estrazione:")
    # for item in os.listdir(extract_dir):
    #     print(f"- {item}")

else:
    print(f"Errore: File zip non trovato al percorso specificato: {zip_path}. Assicurati che il file sia presente e il percorso corretto.")

# Ora i file best.pt e last.pt (se presenti nello zip) dovrebbero trovarsi in /content/runs/detect/OLD_TRAIN/weights/

In [ ]:
#@title Carica il modello YOLOv8 con i pesi addestrati
# Questa cella carica un modello YOLOv8 utilizzando il file dei pesi addestrati
# che è stato estratto dallo zip nella cella precedente (best.pt).
# Questo oggetto 'model' sarà poi utilizzato per eseguire l'inferenza (predizioni)
# su nuove immagini.
#carica modello OLD_TRAIN e testa su nuove scan per vedere i risultati prima a occhio
from ultralytics import YOLO

# Carica il modello specificando il percorso al file dei pesi
#model = YOLO("/content/runs/detect/OLD_TRAIN/weights/best.pt")
MODEL=YOLO("/content/drive/MyDrive/Pesi/best.pt")
model=YOLO(MODEL)
print(f"Modello YOLOv8 caricato dai pesi: /content/runs/detect/OLD_TRAIN/weights/best.pt")

In [ ]:
#@title Funzione per eseguire l'inferenza su una scansione completa e ottenere le bounding box predette 1.0; devi avere jpg in content-test
# Questa cella definisce una funzione chiamata `PassaScansione` che esegue l'inferenza
# (cioè, la predizione delle bounding box) su una singola immagine di scansione completa
# utilizzando il modello YOLOv8 caricato.
# Restituisce l'immagine originale e un array NumPy contenente le coordinate delle bounding box predette.
# Successivamente, la cella itera su tutti i file JPG nella cartella /content/ ed applica questa funzione a ciascuno.
# Il risultato è che le variabili 'img' e 'boxes' contengono i risultati dell'inferenza SOLO per l'ULTIMO file JPG processato.
# La lista 'bb' raccoglie il numero di box trovate per ogni immagine, e viene stampato il totale.
#Versione originale 1.0
#test su scansione 1A ingresso 1
#questo script inference su tuttel le jpg nel content; però sovrascrive i dati di tutte; quindi rimangono solo le ultime?
def PassaScansione(path_scan):
    """
    Esegue l'inferenza su una scansione completa e restituisce l'immagine e le box predette.

    Args:
        path_scan (str): Percorso al file immagine della scansione.

    Returns:
        tuple: (immagine_letta, array_bounding_box)
               array_bounding_box è in formato [x1, y1, x2, y2] in pixel.
    """
    full_scan = cv2.imread(path_scan) # Legge l'immagine
    if full_scan is None:
        print(f"Errore: impossibile caricare l'immagine da {path_scan}")
        return None, np.array([]) # Restituisce None e array vuoto in caso di errore

    H, W, _ = full_scan.shape
    print(f"Dimensioni scansione: {W} x {H}")

    # Esegue la predizione con il modello.
    # imgsz=full_scan.shape[:2] imposta la dimensione di inferenza alla dimensione nativa dell'immagine.
    # conf=0.5 imposta la soglia di confidenza minima per le predizioni.
    # verbose=False disattiva l'output dettagliato di YOLO durante la predizione.
    # Nota: Questa versione restituisce solo le box xyxy e NON i punteggi di confidenza.
    # I punteggi sono necessari per NMS, quindi questa funzione andrebbe modificata per restituirli.
    results = model.predict(full_scan, imgsz=full_scan.shape[:2], conf=0.5, verbose=False)

    # Estrae le coordinate delle bounding box predette in formato [x1, y1, x2, y2]
    # .cpu().numpy() sposta i risultati da un eventuale device GPU/TPU alla CPU e li converte in un array NumPy.
    boxes = results[0].boxes.xyxy.cpu().numpy()

    print(f"Totale fori predetti: {len(boxes)}")
    return full_scan, boxes # Restituisce l'immagine e le box

# Itera su tutti i file .jpg nella cartella /content
# **ATTENZIONE**: Questo loop sovrascrive le variabili 'img' e 'boxes' ad ogni iterazione,
# quindi alla fine conterranno solo i risultati dell'ultimo file processato.
# Per processare e salvare i risultati di tutte le immagini, sarebbe meglio
# salvare i risultati in un dizionario o una lista.
bb=[] # Lista per contare le box per ogni file
print("Esecuzione inferenza su file JPG in /content/...")
for file in os.listdir("/content"):
    if file.lower().endswith(".jpg"):
        file_path = os.path.join("/content", file)
        # Chiama la funzione per eseguire l'inferenza
        img, boxes = PassaScansione(file_path)
        # Aggiunge il conteggio delle box alla lista
        bb+=[len(boxes)]

# Stampa il numero totale di box trovate su tutte le immagini processate nel loop
# (basato sulla lista 'bb').
print(f"\nConteggio totale box su tutti i file processati: {np.sum(bb)}")

# Dopo questo loop, 'img' e 'boxes' contengono i risultati SOLO dell'ultima immagine in /content.
# Le celle successive che usano 'img' e 'boxes' lavoreranno solo sull'ultima immagine.

In [ ]:
#@title Visualizzazione delle bounding box predette sull'immagine della scansione completa 1.0 prima scan -necessita jpg in contentCOLAB
# Questa cella visualizza le bounding box predette sull'immagine.
# **ATTENZIONE**: Utilizza le variabili 'img' e 'boxes' che, a causa del loop nella cella precedente,
# contengono i risultati SOLO dell'ultima immagine processata.
# Quindi, questa cella visualizzerà i risultati solo per l'ultima immagine trovata in /content/
# al momento dell'esecuzione della cella 8O6omM2PmL-H.
# Per visualizzare i risultati di tutte le immagini, sarebbe necessario iterare sui risultati salvati
# in un dizionario (come fatto nella cella 8e36cd90).
import cv2
import matplotlib.pyplot as plt
import os # Importato per os.path.basename nel titolo

# Assicurati che 'img' e 'boxes' siano state popolate dalla cella precedente
if 'img' in globals() and 'boxes' in globals() and img is not None and boxes is not None:
    # Crea una copia dell'immagine per disegnare le box
    out_img = img.copy()

    # Disegna un rettangolo per ogni bounding box trovata
    for box in boxes:
          # Estrai le coordinate (possono essere float, converti in int per disegnare)
          x_min, y_min, x_max, y_max = box.astype(int)
          # Disegna il rettangolo sull'immagine (colore verde BGR: (0, 255, 0), spessore 2)
          cv2.rectangle(out_img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Visualizza l'immagine con le bounding box disegnate
    plt.figure(figsize=(12, 12))
    # Matplotlib usa formato RGB, mentre OpenCV usa BGR, quindi convertiamo
    plt.imshow(cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB))
    # Aggiunge un titolo al grafico
    # Se img_path fosse ancora disponibile, si potrebbe usare il nome del file originale.
    # Poiché non lo è, il titolo è generico.
    plt.title(f"Scansione intera - Fori rilevati: {len(boxes)}")
    plt.axis("off") # Nasconde gli assi
    plt.show()
else:
    print("Immagine o bounding box non disponibili. Esegui prima la cella di inferenza (8O6omM2PmL-H).")

In [ ]:
#@title # Salvataggio delle versioni delle librerie installate in un file di testo
# Questa cella (commentata) mostra il comando per generare un file requirements.txt
# contenente le versioni esatte delle librerie specificate (numpy, torch, ultralytics, opencv-python, albumentations)
# installate nell'ambiente corrente.
# Questo file è utile per ricreare lo stesso ambiente in futuro, garantendo la riproducibilità dei risultati.
# Il comando `pip freeze` elenca tutte le librerie installate, e `grep -E '...'` filtra solo quelle desiderate.
# `>` reindirizza l'output al file specificato.
#!pip freeze | grep -E '^(numpy|torch|ultralytics|opencv-python|albumentations)' \
#    > freeze_cpu_2025-06-26.txt
# La cella 51efa780 utilizza un file simile salvato su Drive per installare le librerie.

In [ ]:
# Download del file dei pesi best.pt e esportazione del modello in formato ONNX
# Questa cella esegue due azioni:
# 1. Scarica il file dei pesi del modello addestrato ('best.pt') sul tuo computer locale.
#    Questo è utile per salvare i pesi addestrati al termine della sessione Colab.
# 2. Esporta il modello addestrato in formato ONNX (Open Neural Network Exchange).
#    ONNX è un formato standard che permette di usare il modello con diversi framework di inferenza (come ONNX Runtime)
#    su diverse piattaforme, potenzialmente ottimizzato per la velocità.
# Assicurati che il modello 'model' sia stato caricato prima di eseguire questa cella.
from google.colab import files

# Tenta di scaricare il file best.pt dal percorso dove Ultralytics lo salva di default dopo il training.
# Il percorso 'runs/detect/train/weights/best.pt' è tipico se il nome del run di training era 'train'.
# Potrebbe essere necessario adattare questo percorso se hai usato un nome di run diverso.
print("Tentativo di download del file best.pt...")
try:
  files.download("runs/detect/train/weights/best.pt")
  print("Download di best.pt avviato.")
except Exception as e:
  print(f"Errore durante il download di best.pt. Assicurati che il file esista al percorso specificato. Errore: {e}")


# Esporta il modello caricato (variabile 'model') in formato ONNX.
# Questo creerà un file .onnx nella stessa directory dove si trova best.pt
if 'model' in globals():
    print("Esportazione del modello in formato ONNX...")
    try:
        model.export(format="onnx")
        print("Esportazione ONNX completata.")
        # Potresti voler scaricare anche il file .onnx
        # files.download("runs/detect/train/weights/best.onnx") # Adatta il percorso
    except Exception as e:
        print(f"Errore durante l'esportazione ONNX. Errore: {e}")
else:
    print("Errore: Modello 'model' non definito. Caricare il modello prima di esportare.")

In [ ]:
#@title Parte successiva del codice Per creazione csv e ordinamento dei file

### Test Inferenza su Nuove Scansioni

Eseguiamo l'inferenza sulle tre scansioni di test specificate utilizzando il modello caricato. Useremo la funzione `PassaScansione` definita in precedenza.

L'inferenza è stata eseguita su ciascuna delle immagini specificate (se trovate). I risultati (l'immagine caricata e le bounding box predette) sono stati memorizzati nel dizionario `inference_results`.

Ora possiamo visualizzare i risultati per ogni immagine per vedere le box rilevate. Creeremo una cella separata per la visualizzazione di ciascun risultato memorizzato.

In [ ]:
# Copia immagini di scansione da Google Drive a una cartella locale in Colab.
# Questa cella crea una cartella `/content/scans` e copia al suo interno
# tutti i file JPG dalla directory specificata sul Google Drive collegato.
# Copiare i dati localmente in Colab (`/content/`) è generalmente più veloce
# rispetto a leggerli direttamente da Google Drive durante l'elaborazione.
# Il comando `!mkdir -p` crea la directory se non esiste.
# Il comando `!cp` copia i file.
!mkdir -p /content/scans/Radio #modificato in Radio, puoi commentare quella parte e torni a l caso originale
#copia da gdrive sul locale colab per velocizzare il lavoro
!cp /content/drive/MyDrive/FileTesi/ScanzioneProviniUsura/Radio/*.jpg /content/scans/Radio
#!cp /content/drive/MyDrive/FileTesi/ScanzioneProviniUsura/Radio/*.jpg /content
print("Immagini di scansione copiate in /content/scans/")

In [ ]:
'''# Esegue l'inferenza YOLOv8 su un set di immagini e salva i risultati.
# Questa cella carica il modello YOLOv8 addestrato e lo utilizza per eseguire
# l'inferenza su tutte le immagini JPG presenti nella cartella `/content/scans`.
# Configura parametri come la soglia di confidenza (`conf`), la soglia di IoU (`iou`),
# e salva i risultati (immagini con box, e file .txt con le coordinate delle box predette).
# imgsz=source.shape[:2] tenta di eseguire l'inferenza alla risoluzione nativa,
# ma YOLO potrebbe comunque ridimensionare internamente.
# save_txt=True è cruciale perché salva le predizioni in file .txt, che vengono poi usati
# nelle celle successive per l'analisi e la visualizzazione.
import cv2 # Importato di nuovo, ma già nella cella 9WUiQ4r8l7aC
from ultralytics import YOLO             # se manca: pip install -U ultralytics

# 1️⃣  importa la libreria (già fatto sopra, ridondante qui)
# from ultralytics import YOLO

# 2️⃣  carica i pesi (il tuo best.pt, o come lo hai chiamato)
# Assicurati che il modello sia già caricato dalla cella TkoP06Jgo48n
if 'model' not in globals():
    print("Caricamento modello...")
    model = YOLO("/content/runs/detect/OLD_TRAIN/weights/best.pt")          # <-- ora 'model' esiste di nuovo
else:
    print("Modello già caricato.")

# source="content/scans" # Imposta la cartella delle immagini di input
source="/content/scans/Radio_noZoom" #"/content/scans" # Corretto il percorso

# Leggere la prima immagine per ottenere le dimensioni, o usare il percorso della cartella come source per predict
# Se source è una cartella, predict itera automaticamente sui file al suo interno.
# imgsz dovrebbe essere impostato per l'inferenza (potrebbe essere un valore fisso o None per usare la dimensione del modello).
# Se si vuole usare la dimensione nativa per ogni immagine, è meglio ciclare sui file e passare il path come source.
# La riga `source=cv2.imread(source)` è problematica perché source diventa un array numpy invece che un percorso.
# Modifichiamo per passare il percorso della cartella direttamente a model.predict.

# 3️⃣  fai l'inferenza (nessun rientro extra!)
RUN = "full_inference_save" # Nome della cartella dove verranno salvati i risultati di questo run
# ATTENTO ALLA CONFIDENZA; TENDE A PRENDERE ANCHE I BORDI
# INOLTRA ATTENTO ALLE DIMENSIONI; YOLO FA UN RESIZE AGGRESSIVE; FORNISCI DIMENSIONE MASSIMA DEL FILE IMMAGINE
print(f"Avvio inferenza su immagini in {source}...")
model.predict(
    source=source,               # Passa la cartella con le .jpg come source
    imgsz=1280, #MEGLIO USRE DIM IMMAGINE [:2] o sballa pixels # Imposta imgsz a una dimensione fissa, o prova a rimuoverlo per usare la dimensione predefinita del modello. Usare la dimensione nativa (H,W) è meglio ma richiede un loop (come in cella XsqBVylpQs_3). Usiamo 1280 come esempio.
    conf=0.79,                   # Soglia di confidenza minima per considerare una predizione (più alta -> meno falsi positivi)
    iou=0.80,                    # Soglia di IoU per la Non-Maximum Suppression interna di YOLO (rimuove box sovrapposte)
    agnostic_nms=True,           # Applica NMS indipendentemente dalla classe (utile con una singola classe)
    project="runs/detect",       # Cartella principale dove salvare i risultati dei run
    name=RUN,                    # Nome specifico per questo run di inferenza (crea una sottocartella in project)
    save_txt=True,               # SALVA LE ANNOTAZIONI PREDETTE IN FORMATO YOLO (.txt)
    save_conf=True,              # Includi i punteggi di confidenza nei file .txt salvati
    save=True,                   # Salva anche le immagini con le box disegnate
    verbose=False                # Sopprimi l'output dettagliato durante l'inferenza
)

print(f"Inferenza completata. Risultati salvati in runs/detect/{RUN}/")
# I file .txt con le predizioni si troveranno in runs/detect/full_inference_save/labels/'''

Part 3 new code to crop order and use gpu

In [ ]:
##ALTERNATIVA PER AVERE VELOCITà RISPETTO A CPU SINGOLA
# Esegue l'inferenza YOLOv8 su immagini singole mantenendo la risoluzione nativa.
# Questa cella è un'alternativa alla precedente. Invece di passare l'intera cartella a `model.predict`,
# itera manualmente su ogni file JPG nella cartella `/content/scans`.
# Per ogni immagine, legge le sue dimensioni native (H, W) e le passa a `imgsz`.
# Questo assicura che l'inferenza venga eseguita alla risoluzione originale dell'immagine,
# potenzialmente migliorando l'accuratezza per oggetti piccoli o dettagli.
# I risultati (file .txt) vengono salvati, e viene creato un summary CSV con il conteggio dei file .txt generati.
#@title Cell 3.1 – Setup & inferenza full-res (TPU/GPU/cpu)
from ultralytics import YOLO
import cv2, glob, os, pandas as pd, torch
import numpy as np # Import numpy

# Determina il device disponibile (GPU/TPU se presente, altrimenti CPU)
device = 0 if torch.cuda.is_available() else "cpu"
print(f"Usando device: {device}")

# Carica il modello addestrato
if 'model' not in globals():
     model = YOLO("/content/weights/best.pt") #"/content/runs/detect/OLD_TRAIN/weights/best.pt") #older
     print("Modello caricato.")
else:
     print("Modello già caricato.")


RUN = "full_native_loop" # Nome del run per salvare i risultati
rows = [] # Lista per raccogliere i dati per il summary CSV

# Itera su ogni file JPG nella cartella delle scansioni (ordinato alfabeticamente)
print(f"Avvio inferenza loop su immagini in /content/scans/...")
for path in sorted(glob.glob("/content/scans/Radio_noZoom/*.jpg")):#"/content/scans/*.jpg")): #before-older:
    print(f"  Processando: {os.path.basename(path)}")
    img = cv2.imread(path)           # Legge l'immagine solo per ottenere H,W
    if img is None:
        print(f"  ⚠️  Errore: impossibile caricare immagine {path}. Salto.")
        continue
    H,W  = img.shape[:2] # Ottiene altezza e larghezza native

    # Esegue la predizione sull'immagine singola passando il PERCORSO
    model.predict(
        source=path,                 # ← passiamo il **percorso** dell'immagine singola
        imgsz=(H,W),                 # ← risoluzione nativa ⇒ nessun resize aggressivo di YOLO
        conf=0.79, iou=0.80, agnostic_nms=True, # Parametri di confidenza e NMS
        save_txt=True, save_conf=True, save=True, # Salva file .txt, confidenza e immagini con box
        project="runs/detect", name=RUN, # Definisce dove salvare i risultati
        device=device, verbose=False # Specifica il device e sopprime output dettagliato
    )

    # Adesso Ultralytics ha scritto direttamente il file .txt di annotazione
    # nella directory corretta (es. runs/detect/full_native_loop/labels/NomeImmagine.txt)
    # Controlla se il file .txt è stato creato e registra il risultato
    txt_path_check = f"runs/detect/{RUN}/labels/{os.path.basename(path)[:-4]}.txt"
    txt_exists = os.path.exists(txt_path_check)
    # Conta il numero di box nel file TXT se esiste, altrimenti 0
    n_boxes = 0
    if txt_exists:
        try:
            # np.loadtxt con ndmin=2 gestisce anche file vuoti correttamente
            arr = np.loadtxt(txt_path_check, ndmin=2)
            # shape[0] dà il numero di righe (box) se arr non è vuoto, altrimenti 0
            n_boxes = arr.shape[0] if arr.size > 0 else 0
        except Exception as e:
            print(f"  ⚠️ Errore leggendo {txt_path_check}: {e}. Conteggio box impostato a 0.")
            n_boxes = 0

    rows.append([os.path.basename(path), n_boxes]) # Aggiunge nome file e conteggio box al summary

# Crea un DataFrame pandas dal summary e lo salva come CSV(added _radio in csv)
pd.DataFrame(rows, columns=["image","n_boxes"])\
  .to_csv("/content/inference_summary_radio.csv", index=False)
print("\n✔️  Run terminata. Conteggio box per immagine salvato in /content/inference_summary_radio.csv")

In [ ]:
# Copia i file di log e summary CSV da Colab a Google Drive.
# Questa cella serve a salvare i risultati dell'inferenza (in questo caso, il summary CSV)
# dalla sessione Colab temporanea al Google Drive, per evitare di perderli.
# Il comando `!cp -r` copia ricorsivamente la directory di output dei run
# (commentato, potenzialmente molto grande) e il file CSV di summary.
#!cp -r runs/detect/full_native_loop*  \
#      /content/drive/MyDrive/yolo_logs/
# Copia il file inference_summary.csv generato dalla cella precedente
!cp /content/inference_summary.csv  \
      /content/drive/MyDrive/yolo_logs/csv/
##salva i file su gdive per il futuro;
print("File inference_summary.csv copiato su Google Drive.")

In [ ]:
# Copia i file CSV contenenti le annotazioni YOLO (predette) da Google Drive a Colab.
# Questa cella esegue l'operazione opposta rispetto alla cella precedente:
# copia i file CSV che hai precedentemente salvato su Google Drive (nella cartella csvs_per_scan)
# nella directory /content/csvs_per_scan/ della sessione Colab corrente.
# Questo è necessario per poter accedere e utilizzare questi file CSV nelle celle successive,
# ad esempio per l'elaborazione, il filtraggio o la visualizzazione delle bounding box predette.
# Questo era uno dei passaggi che ti servivano per ricreare i box sulle immagini per il confronto visivo.
# Assicurati che il percorso Drive sia corretto e che la cartella esista.
!mkdir -p /content/csvs_per_scan/ # Crea la cartella di destinazione se non esiste
!cp /content/drive/MyDrive/csvs_per_scan/*.csv /content/csvs_per_scan/ # Copia tutti i file .csv

print("File CSV copiati da Google Drive a /content/csvs_per_scan/")

In [ ]:
# Crea un file zip dei risultati di un run di inferenza per il download.
# Questa cella impacchetta i contenuti di una specifica directory di run di inferenza YOLO
# (identificata dal nome del run, es. `full_native_loop`) in un file zip.
# Questo file zip può poi essere scaricato sul tuo computer locale o spostato facilmente.
# È un modo pratico per salvare tutti i risultati di un run (immagini con box, file txt, ecc.)
# in un unico file compresso.
#@title 3.2 creazione zip per passare i file di yololog
# Crea un file zip chiamato full_native_loops.zip contenente la directory runs/detect/full_native_loop*
# L'asterisco '*' qui potrebbe causare problemi se ci sono più cartelle che iniziano con full_native_loop.
# Sarebbe meglio specificare il nome esatto del run se è fisso (es. full_native_loop).
!zip -r /content/full_native_loops.zip  runs/detect/full_native_loop*
#scarica i loop inference sul pc in zip
#aletrnativa 2 al tutto
'''import shutil, os
src_dir = "runs/detect/full_native_loop3*"
shutil.make_archive(base_name="/content/full_native_loops",
                    format="zip",
                    root_dir=os.path.dirname(src_dir),
                    base_dir=os.path.basename(src_dir))
'''
print("File zip dei risultati di inferenza creato in /content/full_native_loops.zip")

#Parte Da riusare in iterazioni future


OK adesso testare veloce nuovo csv se funziona si applica su altre scan e si ordina con kmeans e si taglia gli stronzoni!!




In [ ]:
!pip install -r /content/drive/MyDrive/progetto_fori_yolo/freeze_cpu_YOLOSCRIPTfori_2025-06-26.txt --quiet

In [ ]:
#@title INFERENZA IMMAGINI sia DNT/Optic; settaggio completo; usare per inferenze future sulle SCAN
#
#pass all scans!!
import cv2
import os
import glob
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
IMG_DIR  = "/content/jpg"#"/content/scans/Radio"#"/content/scans/Radio"#vecchie per scan ottiche#"/content/scan_ruotate_da_inferire_ingressi"#"/content/drive/MyDrive/Scan_Orientate"#"/content/scans"
CSV_DIR  = "NUOVE_CSV_RADIO"#"/content/scansCSV_oriented"
os.makedirs(CSV_DIR, exist_ok=True)
MODEL = YOLO("/content/drive/MyDrive/Pesi/best.pt") #pezzo precedente, adesso richiamo dal cloud diretto #/content/PESI/best.pt")#"/content/weights/best.pt")   # <- path you used

for img_path in sorted(glob.glob(f"{IMG_DIR}/*.jpg")):
    H,W = cv2.imread(img_path).shape[:2]
    res = MODEL.predict(img_path, imgsz=(H,W), conf=0.79,
                        iou=0.80,
                        )[0]#verbose=False)[0]
    if not len(res.boxes):
        print("⚠️  no boxes in", img_path);  continue

    df = pd.DataFrame(res.boxes.xyxy.cpu().numpy().astype(int),
                      columns=["x1","y1","x2","y2"])
    df.insert(0, "conf", res.boxes.conf.cpu().numpy())
    stem = Path(img_path).stem
    df.to_csv(f"{CSV_DIR}/{stem}.csv", index=False)
    print("•", stem, "→ CSV ok")
print("🎉  all scans processed →", CSV_DIR)


#parte ingresso/iniziale

In [ ]:
#@title serve a creare un pandas dataframe in coordinate assolute e non relative di yolo
# res.boxes.xyxy is already **absolute pixel coords**  (x1,y1,x2,y2)
boxes_xyxy = res.boxes.xyxy.cpu().numpy().astype(int)        # (N,4)
scores      = res.boxes.conf.cpu().numpy()                   # (N,)
print(f"model returned {len(boxes_xyxy)} boxes   conf∈[{scores.min():.2f},{scores.max():.2f}]")


In [ ]:
#@title questa cella segue ordine bustrofedico a colonna(ingresso); boustrophedon path ↑→↓→ (uscite sono inverse)
##possible fix order cluster kmenas
# Cell 5: Batch‐order each scan’s CSV via serpentine K-Means
##
import glob
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

def order_holes(df, n_cols=None):
    """Adds 'hole_idx' to df by left→right snake ordering on x1,x2,y1,y2 (absolute pixels)."""
    df = df.reset_index(drop=True)
    # 1) compute each bbox’s center‐x
    xc = ((df.x1 + df.x2) / 2).to_numpy().reshape(-1, 1)
    # 2) auto‐estimate how many vertical columns if not given
    if n_cols is None:
        step = (xc.max() - xc.min()) / 20
        n_cols = len(np.unique((xc // step).astype(int)))
    # 3) cluster center-x's into columns
    km = KMeans(n_clusters=n_cols, n_init=10, random_state=42).fit(xc)
    cols = np.argsort(km.cluster_centers_.ravel())  # left→right ordering of clusters
    # 4) assign snake indices
    hole_idx = {}
    counter = 1
    for j, col in enumerate(cols):
        # get all indices in this column
        idxs = np.where(km.labels_ == col)[0]
        # sort within‐column: bottom→top on even columns, top→bottom on odd
        idxs_sorted = sorted(
            idxs,
            key=lambda i: -((df.y1.iat[i] + df.y2.iat[i]) / 2)
                          if (j % 2 == 0)
                          else  ((df.y1.iat[i] + df.y2.iat[i]) / 2)
        )
        for i in idxs_sorted:
            hole_idx[i] = counter
            counter += 1
    df['hole_idx'] = df.index.map(hole_idx)
    return df

# ────────── 2) Loop over all your raw scan CSVs ──────────
IN_DIR  = "/content/NUOVE_CSV_RADIO"#"/content/CSV_toOrder" ###RADIO ## usato per test e ottici"/content/CSV_5"#"/content/SCAN_5"#NUOVE_CSV"#"/content/scansCSV_oriented"         # ← your YOLO‐exported CSVs
OUT_DIR = "/content/NUOVE_CSV/Ordered_radio" ###RADIO ## usato per test e ottici"/content/CSV_5/ORDINATE_CSV"#"/content/ordered_csv_NEW"          # ← will receive *_ordered.csv
os.makedirs(OUT_DIR, exist_ok=True)

for csv_path in sorted(glob.glob(f"{IN_DIR}/*.csv")):
    df = pd.read_csv(csv_path)            # expects columns: conf, x1, y1, x2, y2
    if df.empty:
        print(f"⚠️ Skipping empty file {os.path.basename(csv_path)}")
        continue

    # snake‐order it (auto‐estimate columns)
    df_ord = order_holes(df, n_cols=None)

    base = os.path.splitext(os.path.basename(csv_path))[0]
    out  = os.path.join(OUT_DIR, f"{base}_ordered_NEW.csv")
    df_ord.to_csv(out, index=False)
    print(f"✅ {base}: wrote {len(df_ord)} holes → {out}")

print("🎉 All done! Check ordered_csv/*.csv for your serpentine‐numbered files.")


In [ ]:
# Cell – Pack & Download CSV
import shutil, os
from pathlib import Path

# === cartelle da zippare ===
RAW_DIR      = Path("/content/NUOVE_CSV_RADIO")          # CSV “grezzi” (input)
ORDERED_DIR  = Path("/content/NUOVE_CSV/Ordered_radio")  # CSV ordinati (output)

# === nomi zip da creare ===
RAW_ZIP      = RAW_DIR.parent / "csv_raw.zip"
ORDERED_ZIP  = ORDERED_DIR.parent / "csv_ordered.zip"

# 1. crea gli zip (sovrascrive se già esistono)
for src, zip_path in [(RAW_DIR, RAW_ZIP), (ORDERED_DIR, ORDERED_ZIP)]:
    if zip_path.exists(): zip_path.unlink()
    shutil.make_archive(zip_path.with_suffix(""), format="zip", root_dir=src)
    print(f"✅ Zip creato: {zip_path}  ({zip_path.stat().st_size/1e6:.1f} MB)")

# 2. se sei in Google Colab, avvia il download automatico
try:
    from google.colab import files
    files.download(str(RAW_ZIP))
    files.download(str(ORDERED_ZIP))
except ModuleNotFoundError:
    print("\n📥 Non sei in Colab: scarica gli zip manualmente dal file-browser.")


In [ ]:
#@title quick visual check OF scan 4 CUCITA; usare post kmeans#Apdapt for a fast VScheck on file of scan 4-5
import cv2, pandas as pd, matplotlib.pyplot as plt

def show_order(img_path, csv_path, color=(0,255,0),dpi=300):
    img = cv2.imread(img_path)
    df  = pd.read_csv(csv_path).sort_values('hole_idx')

    H = img.shape[0]
    fs = max(1.2, H/2500)           # font-scale proporzionale (~2.0 su 5 k px)
    thk= int(fs*2)

    for _, r in df.iterrows():
        x1,y1,x2,y2 = map(int,(r.x1,r.y1,r.x2,r.y2))
        n = int(r.hole_idx)
        cv2.rectangle(img,(x1,y1),(x2,y2),color,2)
        cv2.putText(img,str(n),(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,
                    fs,color,thk,cv2.LINE_AA)

    plt.figure(figsize=(18,8))
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB)); plt.axis('off')
    plt.title(Path(csv_path).name); plt.show()

show_order("/content/scan5_shift_0610.jpg","/content/NUOVE_CSV_RADIO/Ordered_radio/scan5_cucita_senza_linea_boustro_cols_Finale_ordered_NEW.csv")
#("/content/scan4_cucita_senza_linea.jpg",
          # "/content/NUOVE_CSV_RADIO/Ordered_radio/scan4_cucita_senza_linea_ordered_NEW.csv")


In [ ]:
#@title ╔════════════  VISUAL CHECK BATCH (ingresso fixed)  ════════════╗
import cv2, glob, os, pandas as pd
from pathlib import Path
from google.colab.patches import cv2_imshow   # solo in Colab

CSV_DIR  = "/content/NUOVE_CSV_RADIO/Ordered_radio" #adattamenteo da uscite#"/content/ordered_csv_fixed2"     # tutti i *_fixed.csv
IMG_DIR  = "/content/scans/Radio" #adattamenteo da uscite#"/content/drive/MyDrive/Scan_Orientate"  # scans .jpg
OUT_DIR  = "/content/Radio_DNT"#adattamenteo da uscite#"/content/overlay_batch"         # salva gli overlay
os.makedirs(OUT_DIR, exist_ok=True)

for csv_path in sorted(Path(CSV_DIR).glob("*.csv")):
    stem = csv_path.stem.replace("_fixed", "").replace("_ordered_NEW","")     # es. T0_90_3B_uscita
    img  = cv2.imread(f"{IMG_DIR}/{stem}.jpg")
    if img is None:
        print("⚠️  immagine mancante:", stem);  continue

    df = pd.read_csv(csv_path)
    for _, r in df.iterrows():
        x1,y1,x2,y2 = map(int, (r.x1,r.y1,r.x2,r.y2))
        idx = int(r.hole_idx)
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(img, str(idx), (x1, y1-6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255,255,255), 1, cv2.LINE_AA)

    out = f"{OUT_DIR}/{stem}_vis.jpg"
    cv2.imwrite(out, img)
    #cv2_imshow(img)     # mostra subito in Colab (facoltativo)
    print("✔ overlay:", out)

print("🎉  Overlay creati per tutte le uscite fixed")
# ╚═════════════════════════════════════════════════════════════╝


In [ ]:
#one click download overlays Radio
#@title serve a  creare un zip completo CSV - overlay controllo - evita di avviare in tempo reale gli overlay
# ╔═══════════════  Prepara ZIP di CSV & overlay ═══════════════╗
import os, glob, zipfile

# ────────── Config ──────────
CSV_DIR   = "/content/NUOVE_CSV_RADIO/Ordered_radio"##"/content/ordered_csv_fixed2"     # i tuoi *_fixed.csv
VIS_DIR   = "/content/Radio_DNT"##"/content/overlay_batch"         # gli overlay che hai appena generato
ZIP_PATH  = "/content/Zip_radio.zip"##"/content/uscite_checksA.zip"     # dove scrivere lo ZIP

# ────────── Rimuovi eventuale vecchio ZIP ──────────
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# ────────── Crea lo ZIP ──────────
with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    # aggiungi CSV
    for f in glob.glob(f"{CSV_DIR}/*.csv"):
        zf.write(f, arcname=os.path.basename(f))
    # aggiungi overlay
    for f in glob.glob(f"{VIS_DIR}/*_vis.jpg"):
        zf.write(f, arcname=os.path.join("overlays", os.path.basename(f)))

print(f"🎉 ZIP creato: {ZIP_PATH}")
# ╚══════════════════════════════════════════════════════╝


In [ ]:
import cv2, os, glob, zipfile, pandas as pd
from pathlib import Path
import re                                           # new

ORDERED_DIR = "/content/NUOVE_CSV/Ordered_radio"#"/content/CSV"#"/content/NUOVE_CSV_RADIO/Ordered_radio"
SCAN_DIR    = "/content/jpg"#"/content/RADIO_SCAN"#"/content/scans/Radio"
PATCH_DIR   = "/content/patches_Radio_carbon"
ZIP_PATH    = "/content/patches_Radio_carbon.zip"
PATCH_SIZE = 512          # lato finale
HS         = PATCH_SIZE//2


os.makedirs(PATCH_DIR, exist_ok=True)
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

global_idx = 1

with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    # grab *all* CSVs and let the code figure out the stem
    for csv_path in sorted(Path(ORDERED_DIR).glob("*.csv")):
        raw_stem = Path(csv_path).stem                 # e.g. Carbon Textile 1A_ordered_NEW
        # strip known trailers (_fixed, _ordered_NEW, _uscita, etc.)
        stem     = re.sub(r"(_fixed|_ordered_NEW|_uscita.*)$", "", raw_stem).rstrip("_")
        img_path = f"{SCAN_DIR}/{stem}.jpg"

        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️  Immagine mancante: {img_path} — salto")
            continue
        H_img, W_img = img.shape[:2]

        df = (pd.read_csv(csv_path)
                .sort_values("hole_idx")               # ensure order
                .reset_index(drop=True))

        scan_outdir = f"{PATCH_DIR}/{stem}"
        os.makedirs(scan_outdir, exist_ok=True)

        for _, r in df.iterrows():
            cx, cy = int((r.x1 + r.x2)//2), int((r.y1 + r.y2)//2)
            x1, y1 = max(0, cx-HS), max(0, cy-HS)
            x2, y2 = min(W_img, cx+HS), min(H_img, cy+HS)
            #patch  = img[y1:y2, x1:x2]
            patch  = img[y1:y2, x1:x2]
            # pad out to exactly 700×700 if near an edge
            h, w = patch.shape[:2]
            h, w   = patch.shape[:2]
            pad_bottom = max(0, PATCH_SIZE - h)
            pad_right  = max(0, PATCH_SIZE - w)

            # se non serve padding, copyMakeBorder è superfluo ma non fa danni
            patch = cv2.copyMakeBorder(
                patch,
                0, pad_bottom,          # top, bottom
                0, pad_right,           # left, right
                borderType=cv2.BORDER_CONSTANT,
                value=0
            )

            fname = f"H{global_idx:03d}_h{int(r.hole_idx):03d}_{stem}.jpg"
            fpath = f"{scan_outdir}/{fname}"
            cv2.imwrite(fpath, patch)
            zf.write(fpath, arcname=f"{stem}/{fname}")

            global_idx += 1

        print(f"✔ {stem}: {len(df)} patch(es))")

print(f"🎉  Ho creato {global_idx-1} patch e le ho zippate in {ZIP_PATH}")


In [ ]:
df[['x1','y1','x2','y2']].describe()


#parte Dedicata alle uscite, interesse per ingressi e RADIO

#SOLO USCITE / ORDINE AND CROP
gruppo A ↓↑↓↑
Le altre
1A, 1B, 2A, 2B, 3B

gruppo B da destra ↑↓↑
6 uscita
,3A uscita

In [ ]:
#@title gruppo B da destra ↑↓↑
import os, glob, cv2, pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
import numpy as np

# ───────────> Config
IN_DIR   = "/content/"
OUT_DIR  = "/content/NEWordered_csv_fixed"
GROUP_B  = {
    "T0_90_3A_uscita",
    "T0_90_6_uscita",
}
os.makedirs(OUT_DIR, exist_ok=True)

# ───────────> Ordering function
def order_holes(df, n_cols=None):
    df = df.reset_index(drop=True)
    xc = ((df.x1 + df.x2)/2).to_numpy().reshape(-1,1)
    if n_cols is None:
        step = (xc.max() - xc.min())/20
        n_cols = len(np.unique((xc//step).astype(int)))
    km   = KMeans(n_clusters=n_cols, n_init=10, random_state=42).fit(xc)
    cols = np.argsort(km.cluster_centers_.ravel())
    hole_idx, counter = {}, 1
    for j, c in enumerate(cols):
        idxs = np.where(km.labels_==c)[0]
        idxs_sorted = sorted(
            idxs,
            key=lambda i: -((df.y1.iat[i]+df.y2.iat[i])/2)
                          if j%2==0
                          else  ((df.y1.iat[i]+df.y2.iat[i])/2)
        )
        for i in idxs_sorted:
            hole_idx[i] = counter
            counter += 1
    df['hole_idx'] = df.index.map(hole_idx)
    return df

# ───────────> Batch: trasforma e ordina
for csv_path in sorted(glob.glob(f"{IN_DIR}/*.csv")):
    base = Path(csv_path).stem.replace("_ordered","")
    # salta quelli non uscita
    if not base.endswith("_uscita"):
        continue

    df = pd.read_csv(csv_path)
    # applichi qui eventuale flip_x/rot180 se serve…
    # df_tf = transform_boxes(df, action, W, H)
    df_tf = df  # se già OK

    df_fixed = order_holes(df_tf)

    # **Solo per Gruppo A**, inverti il count
    if base in GROUP_A:
        total = len(df_fixed)
        df_fixed['hole_idx'] = (total + 1) - df_fixed['hole_idx']

    out_csv = os.path.join(OUT_DIR, f"{base}_fixed.csv")
    df_fixed.to_csv(out_csv, index=False)
    print(f"✅ {base}: fixed saved → {out_csv}")

print("🎉 Tutti i CSV di uscita sono stati ri‐indicizzati correttamente.")


In [ ]:
#@title Cell: correzione batch uscite con serpentine top-down per Gruppo A
import os, glob, cv2, pandas as pd, numpy as np
from pathlib import Path
from sklearn.cluster import KMeans

# ────────── Parametri ──────────
IN_DIR    = "/content"#"/content"              # CSV ordered “sbagliati”
OUT_DIR   = "/content/ordered_csv_fixed2"       # CSV fixed
VIS_DIR   = "/content/OVERLAY_USCITE_FIXED2"    # overlay visivi
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)

# i soli stem di scan uscite che vogliamo partire dall'alto
GROUP_A = {
    "T0_90_1A_uscita",
    "T0_90_1B_uscita",
    "T0_90_2A_uscita",
    "T0_90_2B_uscita",
    "T0_90_3B_uscita",
}

# ────────── Funzione di ordinamento serpentine ──────────
def order_holes(df, n_cols=None, start_from_top: bool = False):
    df = df.reset_index(drop=True)
    xc = ((df.x1 + df.x2)/2).to_numpy().reshape(-1,1)
    if n_cols is None:
        step = (xc.max()-xc.min())/20
        n_cols = len(np.unique((xc//step).astype(int)))
    km = KMeans(n_clusters=n_cols, n_init=10, random_state=42).fit(xc)
    cols = np.argsort(km.cluster_centers_.ravel())
    hole_idx = {}
    counter = 1
    for j, c in enumerate(cols):
        idxs = np.where(km.labels_==c)[0]
        if start_from_top:
            # colonna pari: top→bottom ; dispari: bottom→top
            idxs_sorted = sorted(
                idxs,
                key=lambda i: ((df.y1.iat[i]+df.y2.iat[i])/2)
                              if (j%2==0)
                              else -((df.y1.iat[i]+df.y2.iat[i])/2)
            )
        else:
            # serpentine classico: pari: bottom→top ; dispari: top→bottom
            idxs_sorted = sorted(
                idxs,
                key=lambda i: -((df.y1.iat[i]+df.y2.iat[i])/2)
                              if (j%2==0)
                              else  ((df.y1.iat[i]+df.y2.iat[i])/2)
            )
        for i in idxs_sorted:
            hole_idx[i] = counter
            counter += 1
    df['hole_idx'] = df.index.map(hole_idx).astype(int)
    return df

# ────────── Loop batch ──────────
for csv_path in sorted(glob.glob(f"{IN_DIR}/*_uscita*.csv")):
    base = Path(csv_path).stem.replace("_ordered","")
    df   = pd.read_csv(csv_path)

    # decidi top‐down o bottom‐up
    start_top = base in GROUP_A

    # riordina
    df_fixed = order_holes(df, n_cols=None, start_from_top=start_top)

    # salva CSV fixed
    out_csv = os.path.join(OUT_DIR, f"{base}_fixed.csv")
    df_fixed.to_csv(out_csv, index=False)

    # overlay per sanity‐check
    img_path = f"/content/drive/MyDrive/Scan_Orientate/{base}.jpg"
    img = cv2.imread(img_path)
    if img is not None:
        vis = img.copy()
        for _, r in df_fixed.iterrows():
            x1,y1,x2,y2 = map(int,(r.x1,r.y1,r.x2,r.y2))
            cv2.rectangle(vis,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.putText(vis, str(r.hole_idx),(x1,y1-6),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
        cv2.imwrite(f"{VIS_DIR}/{base}_vis.jpg", vis)

    print(f"✅ {base}: fixed.csv → {out_csv}  "
          f"({'top→down' if start_top else 'bottom→up'})")

print("🎉 Tutto fatto! Controlla:", OUT_DIR, "e", VIS_DIR)


In [ ]:
#@title ╔════════════  VISUAL CHECK BATCH -ottico-(uscite fixed)  ════════════╗
import cv2, glob, os, pandas as pd
from pathlib import Path
from google.colab.patches import cv2_imshow   # solo in Colab

CSV_DIR  = "/content/ordered_csv_fixed2"     # tutti i *_fixed.csv
IMG_DIR  = "/content/drive/MyDrive/Scan_Orientate"  # scans .jpg
OUT_DIR  = "/content/overlay_batch"         # salva gli overlay
os.makedirs(OUT_DIR, exist_ok=True)

for csv_path in sorted(Path(CSV_DIR).glob("*_fixed.csv")):
    stem = csv_path.stem.replace("_fixed", "")     # es. T0_90_3B_uscita
    img  = cv2.imread(f"{IMG_DIR}/{stem}.jpg")
    if img is None:
        print("⚠️  immagine mancante:", stem);  continue

    df = pd.read_csv(csv_path)
    for _, r in df.iterrows():
        x1,y1,x2,y2 = map(int, (r.x1,r.y1,r.x2,r.y2))
        idx = int(r.hole_idx)
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(img, str(idx), (x1, y1-6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255,255,255), 1, cv2.LINE_AA)

    out = f"{OUT_DIR}/{stem}_vis.jpg"
    cv2.imwrite(out, img)
    #cv2_imshow(img)     # mostra subito in Colab (facoltativo)
    print("✔ overlay:", out)

print("🎉  Overlay creati per tutte le uscite fixed")
# ╚═════════════════════════════════════════════════════════════╝


In [ ]:
#@title serve a  creare un zip completo CSV - overlay controllo(ottico-uscita) - evita di avviare in tempo reale gli overlay
# ╔═══════════════  Prepara ZIP di CSV & overlay ═══════════════╗
import os, glob, zipfile

# ────────── Config ──────────
CSV_DIR   = "/content/ordered_csv_fixed2"     # i tuoi *_fixed.csv
VIS_DIR   = "/content/overlay_batch"         # gli overlay che hai appena generato
ZIP_PATH  = "/content/uscite_checksA.zip"     # dove scrivere lo ZIP

# ────────── Rimuovi eventuale vecchio ZIP ──────────
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# ────────── Crea lo ZIP ──────────
with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    # aggiungi CSV
    for f in glob.glob(f"{CSV_DIR}/*.csv"):
        zf.write(f, arcname=os.path.basename(f))
    # aggiungi overlay
    for f in glob.glob(f"{VIS_DIR}/*_vis.jpg"):
        zf.write(f, arcname=os.path.join("overlays", os.path.basename(f)))

print(f"🎉 ZIP creato: {ZIP_PATH}")
# ╚══════════════════════════════════════════════════════╝


In [ ]:

import cv2, os, glob, zipfile, pandas as pd
from pathlib import Path

# ────────── Parametri ──────────
ORDERED_DIR = "/content/ordered_csv_fixed2"        # tutti *_uscita_fixed.csv che hai già sistemato
SCAN_DIR    = "/content/drive/MyDrive/Scan_Orientate"
PATCH_DIR   = "/content/patches_uscite_all"         # output dei crop
ZIP_PATH    = "/content/patches_uscite_all.zip"
HS          = 350                                   # mezzo lato del crop (700×700)

# preparo cartelle
os.makedirs(PATCH_DIR, exist_ok=True)
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

global_idx = 1  # unico contatore globale

with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    # prendo TUTTI i CSV di uscita (sia quelli “_uscita_fixed.csv” sia eventuali varianti)
    for csv_path in sorted(glob.glob(f"{ORDERED_DIR}/*_uscita*_fixed.csv")):
        stem      = Path(csv_path).stem.replace("_fixed","")  # es. "T0_90_1A_uscita"
        img_path  = f"{SCAN_DIR}/{stem}.jpg"
        df        = pd.read_csv(csv_path)
        img       = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Immagine mancante: {img_path}, salto"); continue
        H_img, W_img = img.shape[:2]

        # assicuro che h (= hole_idx) parta da 1 e segua l’ordine corretto
        df = df.sort_values("hole_idx")

        # cartella di output per questa scan
        scan_outdir = f"{PATCH_DIR}/{stem}"
        os.makedirs(scan_outdir, exist_ok=True)
        # ciclo sui fori
        for _, r in df.iterrows():
            cx, cy = int((r.x1 + r.x2)//2), int((r.y1 + r.y2)//2)
            x1, y1 = max(0, cx-HS), max(0, cy-HS)
            x2, y2 = min(W_img, cx+HS), min(H_img, cy+HS)
            patch  = img[y1:y2, x1:x2]
            h, w   = patch.shape[:2]
            # padding fino a esatto 700×700
            patch  = cv2.copyMakeBorder(patch, 0, 700-h, 0, 700-w,
                                        cv2.BORDER_CONSTANT, 0)

            # nome: H<globale>_h<locale>_<scan>.jpg
            fname = f"H{global_idx:03d}_h{int(r.hole_idx):03d}_{stem}.jpg"
            fpath = f"{scan_outdir}/{fname}"
            cv2.imwrite(fpath, patch)
            # aggiungo al ZIP mantenendo la sottocartella per scan
            zf.write(fpath, arcname=f"{stem}/{fname}")

            global_idx += 1

print(f"🎉 Ho creato {global_idx-1} patch e le ho zippate in {ZIP_PATH}")


#PARTE CHE FORSE POTREBBE ESSERE TOLTA ADESSO; teniamo per sicurezza futura, erano celle di test veloci

In [ ]:
#@title --- 0.  install / import once ----------------------------------------------
#!pip install -qqU ultralytics                   # Colab: fetch latest build
from ultralytics import YOLO
import cv2, pandas as pd
from google.colab.patches import cv2_imshow

# --- 1.  load your weight ----------------------------------------------------
MODEL = YOLO("/content/weights/best.pt")   # <- path you used

# --- 2.  choose ONE image ----------------------------------------------------
IMG  = "/content/scans/T0_90_1A_uscita.jpg"                      # test file
img = cv2.imread(IMG)
H,W  = img.shape[:2] #
# --- 3.  predict (no save_txt, no tricks: we only need the returned object) --
res = MODEL.predict(IMG, imgsz=(H,W), conf=0.79, iou=0.80, verbose=False)[0]

# res.boxes.xyxy is already **absolute pixel coords**  (x1,y1,x2,y2)
boxes_xyxy = res.boxes.xyxy.cpu().numpy().astype(int)        # (N,4)
scores      = res.boxes.conf.cpu().numpy()                   # (N,)
print(f"model returned {len(boxes_xyxy)} boxes   conf∈[{scores.min():.2f},{scores.max():.2f}]")


In [ ]:
#@title visual sanity that works on ordered amd shows orders of holes and indexes on one scan
##VISUAL SANITY CHECK WORKS on one scan 4 here: works
import cv2, pandas as pd
from pathlib import Path
from google.colab.patches import cv2_imshow   # solo in Colab

# 1️⃣  scegli il CSV ordinato da visualizzare
CSV_PATH = "/content/ordered_csv_fixed/T0_90_3B_uscita_fixed.csv"#"/content/CSV_5/ORDINATE_CSV/T0_90_5_entrata_NEW_ordered_NEW.csv"##"/content/ORDINATE_CSV/T0_90_3B_ingresso_ordered_NEW.csv"#"/content/ordered_csv_NEW/T0_90_4_ingresso_ordered_NEW.csv"#"/content/T0_90_4_uscita_NEW_ordered.csv" #"/content/ordered_csv_NEW/T0_90_4_uscita_NEW_ordered.csv"

# 2️⃣  leggi il file → DataFrame
df = pd.read_csv(CSV_PATH)          # colonne: conf,x1,y1,x2,y2,hole_idx  (o simile)

# 3️⃣  recupera il percorso dell’immagine dalla stringa “scan” contenuta nel nome file
#     (p.es. T0_90_1A_uscita.jpg si trova in /content/scans/)
img = cv2.imread("/content/drive/MyDrive/Scan_Orientate/T0_90_3B_uscita.jpg")#"/content/SCAN_5/T0_90_5_entrata_NEW.jpg")#"/content/scan_ruotate_da_inferire_ingressi/T0_90_3B_ingresso.jpg")#"/content/drive/MyDrive/Scan_Orientate/T0_90_4_ingresso.jpg")#"/content/drive/MyDrive/FileTesi/ScanzioneProviniUsura/T0_90_4_uscita_NEW.jpg")

#img = cv2.imread(IMG_PATH)
assert img is not None, f"Immagine non trovata: {IMG_PATH}"

# 4️⃣  disegna per ogni riga del DataFrame
for _, r in df.iterrows():
    x1, y1, x2, y2 = int(r.x1), int(r.y1), int(r.x2), int(r.y2)
    idx            = int(r.hole_idx)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, str(idx),
                (x1, y1 - 8),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (255, 255, 255), 1, cv2.LINE_AA)

cv2_imshow(img)     # mostra in Colab


In [ ]:
#@title visual sanity that works on ordered amd shows orders of holes and indexes on one scan
##VISUAL SANITY CHECK WORKS on one scan 4 here: works
import cv2, pandas as pd
from pathlib import Path
from google.colab.patches import cv2_imshow   # solo in Colab

# 1️⃣  scegli il CSV ordinato da visualizzare
CSV_PATH = "/content/ordered_csv_fixed/T0_90_3A_uscita_fixed.csv"#"/content/CSV_5/ORDINATE_CSV/T0_90_5_entrata_NEW_ordered_NEW.csv"##"/content/ORDINATE_CSV/T0_90_3B_ingresso_ordered_NEW.csv"#"/content/ordered_csv_NEW/T0_90_4_ingresso_ordered_NEW.csv"#"/content/T0_90_4_uscita_NEW_ordered.csv" #"/content/ordered_csv_NEW/T0_90_4_uscita_NEW_ordered.csv"

# 2️⃣  leggi il file → DataFrame
df = pd.read_csv(CSV_PATH)          # colonne: conf,x1,y1,x2,y2,hole_idx  (o simile)

# 3️⃣  recupera il percorso dell’immagine dalla stringa “scan” contenuta nel nome file
#     (p.es. T0_90_1A_uscita.jpg si trova in /content/scans/)
img = cv2.imread("/content/drive/MyDrive/Scan_Orientate/T0_90_3A_uscita.jpg")#"/content/SCAN_5/T0_90_5_entrata_NEW.jpg")#"/content/scan_ruotate_da_inferire_ingressi/T0_90_3B_ingresso.jpg")#"/content/drive/MyDrive/Scan_Orientate/T0_90_4_ingresso.jpg")#"/content/drive/MyDrive/FileTesi/ScanzioneProviniUsura/T0_90_4_uscita_NEW.jpg")

#img = cv2.imread(IMG_PATH)
assert img is not None, f"Immagine non trovata: {IMG_PATH}"

# 4️⃣  disegna per ogni riga del DataFrame
for _, r in df.iterrows():
    x1, y1, x2, y2 = int(r.x1), int(r.y1), int(r.x2), int(r.y2)
    idx            = int(r.hole_idx)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, str(idx),
                (x1, y1 - 8),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (255, 255, 255), 1, cv2.LINE_AA)

cv2_imshow(img)     # mostra in Colab

## creazione del check visivo

In [ ]:
#@title Cell: evaluate serpentine ordering with quantitative adjacency metrics
import glob
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

def check_serpentine(df_ordered):
    # 1) compute centers
    df = df_ordered.copy()
    df['cx'] = (df.x1 + df.x2) / 2
    df['cy'] = (df.y1 + df.y2) / 2

    # 2) re-cluster into columns exactly as before
    X = df['cx'].to_numpy().reshape(-1,1)
    # auto n_cols
    step = (X.max() - X.min()) / 20
    n_cols = len(np.unique((X // step).astype(int)))
    km = KMeans(n_clusters=n_cols, n_init=10, random_state=42).fit(X)
    df['col'] = km.labels_

    # 3) sort by hole_idx to walk the snake
    df = df.sort_values('hole_idx').reset_index(drop=True)

    # 4) for each i → i+1, check adjacency rule
    correct = []
    for i in range(len(df)-1):
        c0, c1 = df.loc[i,'col'], df.loc[i+1,'col']
        y0, y1 = df.loc[i,'cy'],  df.loc[i+1,'cy']
        x0, x1 = df.loc[i,'cx'],  df.loc[i+1,'cx']

        if c0 == c1:
            # within same column: even‐indexed columns (0‐based) go bottom→top (y increases),
            # odd‐indexed go top→bottom (y decreases)
            if (c0 % 2 == 0 and (y1 - y0) > 0) or (c0 % 2 == 1 and (y1 - y0) < 0):
                correct.append(True)
            else:
                correct.append(False)
        else:
            # between columns must always move rightwards (x increases)
            correct.append((x1 - x0) > 0)

    # 5) return fraction correct + total steps
    return np.mean(correct), len(correct)

# 6) batch over all ordered CSVs
RESULTS = []
for path in sorted(glob.glob("/content/ordered_csv_NEW/*.csv")):
    df_ord = pd.read_csv(path)
    frac, steps = check_serpentine(df_ord)
    name = path.split("/")[-1].replace("_ordered_NEW.csv","")
    RESULTS.append((name, frac, steps))

# assemble a quick table
res_df = pd.DataFrame(RESULTS, columns=["scan","frac_correct","n_steps"])
print(res_df.to_markdown(index=False))

# highlight any scans <100%:
bad = res_df[res_df.frac_correct < 1.0]
if not bad.empty:
    print("\n⚠️ These scans have mis-ordered adjacencies:")
    print(bad.to_markdown(index=False))
else:
    print("\n✅ All scans follow a perfect serpentine path!")


##end here

In [ ]:
IMG_DIR  = "/content/SCAN_5"#"/content/drive/MyDrive/Scan_Orientate"#"/content/scans"
CSV_DIR  = "/content/CSV_5"#"/content/scansCSV_oriented"
os.makedirs(CSV_DIR, exist_ok=True)
MODEL = YOLO("/content/PESI/best.pt")#"/content/weights/best.pt")   # <- path you used

for img_path in sorted(glob.glob(f"{IMG_DIR}/*.jpg")):
    H,W = cv2.imread(img_path).shape[:2]
    res = MODEL.predict(img_path, imgsz=(H,W), conf=0.79,
                        iou=0.80,
                        )[0]#verbose=False)[0]
    if not len(res.boxes):
        print("⚠️  no boxes in", img_path);  continue

    df = pd.DataFrame(res.boxes.xyxy.cpu().numpy().astype(int),
                      columns=["x1","y1","x2","y2"])
    df.insert(0, "conf", res.boxes.conf.cpu().numpy())
    stem = Path(img_path).stem
    df.to_csv(f"{CSV_DIR}/{stem}.csv", index=False)
    print("•", stem, "→ CSV ok")
print("🎉  all scans processed →", CSV_DIR)


In [ ]:
#@title -ONE TO USE- crops patchees NEW, avoid uscita 6 e odina sottocartelle scan
##Skips every CSV whose name contains _6_uscita (until you fix its order). 1.
"""patches_700/
  ├── T0_90_4_ingresso/
  │     ├── H257_h032_T0_90_4_ingresso.jpg
  │     └── …
  └── T0_90_4_uscita/
        └── …
"""
# Adds a dual index in the file-name: -- H<GLOBAL:03d>_h<SCANIDX:03d>_<scan_name>.jpg
'''where GLOBAL is a running counter over all patches (so H000, H001 … H599) and SCANIDX is the per-scan hole_idx.'''
# Cell ▸ crop patches into per-scan sub-folders with dual index ──────────────
# crop patches into per-scan sub-folders – with ordered rows
#adattato per RADIO basta inserire in una folder diversi csv ordered e in una
#METTERE DUE CARTELLE, se FILE SINGOLI SI BLOCCA, LAVORA SU GRUPPI DI FILE
import cv2, os, glob, zipfile, pandas as pd
from pathlib import Path

ORDERED_DIR = "/content/CSV"#"/content/CSV_5/ORDINATE_CSV"#"/content/ORDINATE_CSV"#"/content"                                   # *_ordered*.csv
SCAN_DIR    = "/content/RADIO_SCAN"#"/content/SCAN_5"#"/content/scan_ruotate_da_inferire_ingressi"#"/content/drive/MyDrive/Scan_Orientate"      # rotated scans
PATCH_DIR   = "/content/Radio/Carbon"#"/content/SCAN_5/PATCHES_5"#"/content/Patches_nuove"#"/content/patches_700_NEW"
ZIP_PATH    = "/content/Patches_Radio_patches.zip"#"/content/Patches_nuove.zip"#"/content/patches_700_NEW.zip"
HS = 256#350                                                   # half-crop size

os.makedirs(PATCH_DIR, exist_ok=True)
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

global_idx = 1

with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as zf:
    for csv_path in sorted(glob.glob(f"{ORDERED_DIR}/*_ordered*.csv")):
        if "_6_uscita" in csv_path:        # still on hold
            print("⏩  skipping", Path(csv_path).name)
            continue

        stem = Path(csv_path).stem
        for suf in ("_ordered_NEW", "_ordered"):
            if stem.endswith(suf):
                stem = stem[:-len(suf)]
                break
        scan_name = stem

        img_path = f"{SCAN_DIR}/{scan_name}.jpg"
        img      = cv2.imread(img_path)
        if img is None:
            print(f"⚠️  missing image {img_path}, skipping"); continue
        H, W = img.shape[:2]

        df = pd.read_csv(csv_path)
        if "manual_flag" in df.columns:
            df = df[df.manual_flag != 1]

        df = df.sort_values("hole_idx")    #  ← ensure 1,2,3…

        scan_outdir = f"{PATCH_DIR}/{scan_name}"
        os.makedirs(scan_outdir, exist_ok=True)

        for _, r in df.iterrows():
            cx = int((r.x1 + r.x2) // 2)
            cy = int((r.y1 + r.y2) // 2)
            x1, x2 = cx - HS, cx + HS
            y1, y2 = cy - HS, cy + HS
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(W, x2), min(H, y2)

            patch = img[y1:y2, x1:x2]
            h, w  = patch.shape[:2]
            patch = cv2.copyMakeBorder(patch, 0, 700-h, 0, 700-w,
                                       cv2.BORDER_CONSTANT, value=0)

            fname = (f"H{global_idx:03d}_"
                     f"h{int(r.hole_idx):03d}_{scan_name}.jpg")
            fpath = f"{scan_outdir}/{fname}"
            cv2.imwrite(fpath, patch)
            zf.write(fpath, arcname=f"{scan_name}/{fname}")
            global_idx += 1

print(f"🎉  Cropped {global_idx-1} patches to {PATCH_DIR}  (ZIP: {ZIP_PATH})")


PER IL FUTURO CROP EASY DIRETTO

In [ ]:
#@title MODIFICA CODICE SOTTO PER RESIZE DELLE PATCH
#CREAA I CROP DELLE RADIO EVITANDO DI INSERIRE FORI VICINI
#Dobbiamo modificare il tuo script di cropping per renderlo "intelligente". Invece di tagliare una finestra di pixel fissa, deve **normalizzare la scala** di ogni patch in modo che il foro al centro abbia sempre la stessa dimensione in pixel.
#
#Ecco la logica:
#1.  **Calcola la Dimensione del Foro:** Per ogni bounding box rilevata da YOLO, calcoliamo la sua dimensione media (es. `(larghezza + altezza) / 2`). Questo ci dà il diametro del foro in pixel nella scansione originale.
#2.  **Definisci una Scala Target:** Decidiamo quale dovrebbe essere la dimensione standard di un foro nelle nostre patch finali. Ad esempio, vogliamo che in una patch finale da 512x512, il foro abbia sempre un diametro di circa 280 pixel.
#3.  **Calcola il Fattore di Scala:** Per ogni foro, confrontiamo la sua dimensione reale in pixel con la nostra dimensione target. Il rapporto `(dimensione_target / dimensione_reale)` ci dà il fattore di scala da applicare.
#4.  **Ritaglia e Ridimensiona:**
#    *   Calcoliamo quanto grande deve essere l'area da ritagliare dall'immagine originale affinché, una volta ridimensionata con il nostro fattore di scala, diventi esattamente 512x512.
#    *   Eseguiamo il ritaglio.
#    *   Applichiamo il ridimensionamento (`cv2.resize`).
#
#Il risultato sarà una cartella di patch dove ogni foro, indipendentemente dalla scansione di provenienza, apparirà della stessa dimensione, pronto per essere processato dalla "Mask Factory v5".

In [ ]:
import cv2
import os
import zipfile
import pandas as pd
from pathlib import Path
import re
import numpy as np

# --- IMPOSTAZIONI ---
CSV_DIR     = "/content/NUOVE_CSV/Ordered_radio"#"/content/csv"
SCAN_DIR    = "/content/jpg"
PATCH_DIR   = "/content/Radio_Patches_Normalized" # Nuovo nome per chiarezza
ZIP_PATH    = "/content/Radio_Patches_Normalized.zip"

# --- NUOVE IMPOSTAZIONI PER LA NORMALIZZAZIONE DELLA SCALA ---
# 1. Dimensione finale della patch in pixel.
FINAL_PATCH_SIZE = 512

# 2. Dimensione TARGET del diametro del foro all'interno della patch finale.
#    Questo è il parametro più importante da sintonizzare. Un buon punto di partenza
#    è poco più della metà della dimensione della patch.
#    Con questo valore, la "Mask Factory v5" dovrà avere raggi di Hough intorno a 140-150.
TARGET_HOLE_DIAMETER_PIXELS = 290

# --- SCRIPT ---

os.makedirs(PATCH_DIR, exist_ok=True)
if os.path.exists(ZIP_PATH):
    print(f"Rimuovo ZIP precedente: {ZIP_PATH}")
    os.remove(ZIP_PATH)

global_idx = 1
print("🚀 Inizio processo di cropping normalizzato...")

with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    csv_files = sorted(Path(CSV_DIR).glob("*.csv"))
    if not csv_files:
        print(f"🛑 ERRORE: Nessun file .csv trovato in {CSV_DIR}. Controlla il percorso.")

    for csv_path in csv_files:
        print(f"\n📄 Processo il file CSV: {csv_path.name}")

        raw_stem = csv_path.stem
        stem = re.sub(r"(_fixed|_ordered_NEW|_ordered)$", "", raw_stem).rstrip("_")
        img_path = Path(SCAN_DIR) / f"{stem}.jpg"

        if not img_path.exists():
            print(f"⚠️  Immagine mancante per '{stem}': {img_path} — Salto.")
            continue

        img = cv2.imread(str(img_path))
        H_img, W_img = img.shape[:2]
        print(f"🖼️  Immagine '{img_path.name}' trovata e caricata ({W_img}x{H_img}).")

        df = pd.read_csv(csv_path)
        # Assicurati che esista la colonna 'hole_idx', altrimenti ordina in base alla posizione y,x
        if 'hole_idx' in df.columns:
            df = df.sort_values("hole_idx").reset_index(drop=True)
        else:
            print("⚠️ Colonna 'hole_idx' non trovata. Ordinamento per y,x.")
            df = df.sort_values(['y1', 'x1']).reset_index(drop=True)
            df['hole_idx'] = df.index + 1

        scan_outdir = Path(PATCH_DIR) / stem
        scan_outdir.mkdir(exist_ok=True)

        for _, r in df.iterrows():
            # --- INIZIO BLOCCO DI LOGICA PER LA NORMALIZZAZIONE ---

            # 1. Calcola il centro e il diametro del foro rilevato in pixel
            cx, cy = int((r.x1 + r.x2) / 2), int((r.y1 + r.y2) / 2)
            w, h = r.x2 - r.x1, r.y2 - r.y1
            detected_diameter = (w + h) / 2

            if detected_diameter < 1: # Evita divisione per zero
                continue

            # 2. Calcola il fattore di scala necessario
            scale_factor = TARGET_HOLE_DIAMETER_PIXELS / detected_diameter

            # 3. Calcola la dimensione dell'area da ritagliare DALL'IMMAGINE ORIGINALE
            #    Questa sarà un'area più grande o più piccola di 512x512 a seconda dello scale_factor
            crop_size_original_img = int(FINAL_PATCH_SIZE / scale_factor)

            # Calcola le coordinate per questo crop più grande
            hs_original = crop_size_original_img // 2
            x1, y1 = cx - hs_original, cy - hs_original
            x2, y2 = cx + hs_original, cy + hs_original

            # 4. Estrai la patch dall'immagine originale (con padding implicito se esce dai bordi)
            #    np.pad gestisce i bordi in modo pulito, evitando errori
            pad_left = abs(min(0, x1))
            pad_top = abs(min(0, y1))
            pad_right = abs(min(0, W_img - x2))
            pad_bottom = abs(min(0, H_img - y2))

            # Estrai la parte valida dell'immagine
            patch_original = img[max(0, y1):min(H_img, y2), max(0, x1):min(W_img, x2)]

            # Applica il padding se il crop usciva dai bordi
            patch_padded = np.pad(
                patch_original,
                ((pad_top, pad_bottom), (pad_left, pad_right), (0, 0)),
                mode='constant',
                constant_values=0 # Bordo nero
            )

            # 5. Ridimensiona la patch estratta alla dimensione finale.
            #    Questa operazione normalizza la scala!
            patch_normalized = cv2.resize(
                patch_padded,
                (FINAL_PATCH_SIZE, FINAL_PATCH_SIZE),
                interpolation=cv2.INTER_LANCZOS4 # Ottima qualità per il down/up-scaling
            )

            # --- FINE BLOCCO DI LOGICA PER LA NORMALIZZAZIONE ---

            fname = f"H{global_idx:03d}_h{int(r.hole_idx):03d}_{stem}.jpg"
            fpath = scan_outdir / fname
            cv2.imwrite(str(fpath), patch_normalized)

            zf.write(fpath, arcname=f"{stem}/{fname}")
            global_idx += 1

        print(f"✔  Creati {len(df)} patch normalizzate per '{stem}'.")

print(f"\n🎉  Processo completato! Create {global_idx-1} patch normalizzate in {PATCH_DIR} e zippate in {ZIP_PATH}")

#Obiettivo: Convertire ogni patch 512x512 in una nuova patch in cui il foro da 6 mm abbia sempre la stessa dimensione in pixel. Questo crea una "scala canonica".
<details>
    Definire una Scala Canonica (Target):

        Hai già notato che i fori originali variano da ~120 a ~300 px. Scegliamo una dimensione target ragionevole, ad esempio TARGET_DIAMETER_PX = 280. Questo valore dovrebbe essere scelto per minimizzare l'upscaling (che può creare artefatti) e il downscaling (che può perdere dettagli del danno). La tua media di 297 px è un'ottima candidata, ma per questo esempio uso 280.

        La tua scala canonica (S_canon) sarà S_canon = TARGET_DIAMETER_PX / 6.0 mm. Questa è la costante px/mm che useremo per tutte le immagini dopo la normalizzazione.

    Processare Ogni Immagine Individualmente (Ciclo for sui 592 fori):

        (a) Misura il diametro del foro attuale: Per ogni patch, esegui la tua funzione (basata su cv2.HoughCircles o simile) per trovare il diametro in pixel del foro (current_diameter_px).

        (b) Calcola il Fattore di Ridimensionamento: Calcola scale_factor = TARGET_DIAMETER_PX / current_diameter_px.

            Se scale_factor > 1, ingrandirai l'immagine.

            Se scale_factor < 1, la rimpicciolirai.

        (c) Ridimensiona l'Immagine e la Maschera: Applica questo scale_factor sia all'immagine del patch sia, se la possiedi già, alla sua maschera di segmentazione del danno. Usa cv2.resize. L'interpolazione cv2.INTER_AREA è buona per rimpicciolire, mentre cv2.INTER_CUBIC o cv2.INTER_LINEAR è migliore per ingrandire. Per le maschere, usa sempre cv2.INTER_NEAREST per non creare valori intermedi (es. 0.5).

        (d) Estrai le Feature Geometriche: Solo ora, sull'immagine normalizzata, esegui il tuo algoritmo per calcolare l'area del danno (A_del_px), il diametro massimo (D_max_px), ecc.

        (e) Converti le Feature in Unità Fisiche (mm): Usa la scala canonica che hai definito.

            A_del [mm^2] = A_del_px / (S_canon)^2

            D_MAX [mm] = D_max_px / S_canon
            Questo è il passaggio chiave che correggerà l'errore fino a ×70. L'errore nasceva perché dividevi un'area in pixel misurata a una scala per un fattore di scala calcolato su un'altra.

    Salva i Risultati:

        Salva le patch normalizzate in una nuova cartella. Saranno l'input per la tua UNet++.

        Salva le feature calcolate (A_del [mm^2], D_MAX [mm], etc.) in un nuovo file CSV, pronte per la LSTM.

In [ ]:
#@title check metrico delle scale e edeelle nuove patches
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import zipfile
import random

# --- CONFIGURAZIONE DELL'ANALISI ---

# Percorso del file ZIP contenente le patch normalizzate
ZIP_TO_ANALYZE = "/content/Radio_Patches_Normalized.zip"

# Directory dove estrarre le patch per l'analisi
PATCHES_DIR_ANALYSIS = "/content/analysis_patches/"

# Il valore target che ci aspettiamo di trovare (per confronto)
TARGET_DIAMETER = 290
FINAL_PATCH_SIZE = 512

print("🚀 Inizio analisi qualitativa della normalizzazione...")

# --- 1. ESTRAZIONE DEI DATI ---
if os.path.exists(PATCHES_DIR_ANALYSIS):
    import shutil
    shutil.rmtree(PATCHES_DIR_ANALYSIS) # Pulisce la directory se esiste già

with zipfile.ZipFile(ZIP_TO_ANALYZE, 'r') as zip_ref:
    zip_ref.extractall(PATCHES_DIR_ANALYSIS)
print(f"✔ Patch estratte in: {PATCHES_DIR_ANALYSIS}")

image_paths = glob.glob(os.path.join(PATCHES_DIR_ANALYSIS, "**/*.jpg"), recursive=True)
print(f"Trovate {len(image_paths)} patch da analizzare.")

# --- 2. MISURAZIONE AUTOMATICA ---
measurements = []

for img_path in image_paths:
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Un leggero blur aiuta HoughCircles a essere più robusto
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # HoughCircles per trovare il foro
    # I parametri sono "sintonizzati" sulla nostra scala normalizzata
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1.2,             # Rapporto di risoluzione dell'accumulatore
        minDist=FINAL_PATCH_SIZE, # Distanza minima tra i centri (ci aspettiamo un solo foro)
        param1=50,          # Soglia superiore per Canny edge detector
        param2=30,          # Soglia dell'accumulatore (più basso = più cerchi trovati)
        minRadius=int(TARGET_DIAMETER / 2 * 0.8), # Raggio min: 80% del raggio target
        maxRadius=int(TARGET_DIAMETER / 2 * 1.2)  # Raggio max: 120% del raggio target
    )

    if circles is not None:
        # Arrotonda e prendi il primo (e unico) cerchio trovato
        circles = np.round(circles[0, :]).astype("int")
        (x, y, r) = circles[0]
        diameter = r * 2
        measurements.append({'path': img_path, 'measured_diameter': diameter})

print(f"✔ Misurati i diametri per {len(measurements)} patch.")

# --- 3. ANALISI STATISTICA CON PANDAS ---
df_analysis = pd.DataFrame(measurements)

print("\n📊 Statistiche Descrittive dei Diametri Misurati:")
print(df_analysis['measured_diameter'].describe())

# --- 4. VISUALIZZAZIONE CON MATPLOTLIB ---
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(12, 6))

# Istogramma
ax.hist(df_analysis['measured_diameter'], bins=30, alpha=0.8, label='Distribuzione Diametri Misurati')
ax.axvline(TARGET_DIAMETER, color='r', linestyle='--', linewidth=2, label=f'Target Diameter ({TARGET_DIAMETER}px)')
ax.axvline(df_analysis['measured_diameter'].mean(), color='g', linestyle=':', linewidth=2, label=f'Media ({df_analysis["measured_diameter"].mean():.2f}px)')

ax.set_title("Distribuzione dei Diametri dei Fori nelle Patch Normalizzate", fontsize=16)
ax.set_xlabel("Diametro Misurato (pixel)", fontsize=12)
ax.set_ylabel("Numero di Patch", fontsize=12)
ax.legend()
plt.show()

# --- 5. VERIFICA VISIVA (SPOT-CHECK) ---
print("\n🖼️ Esempi di verifica visiva (patch con cerchio rilevato e diametro):")

# Seleziona 9 campioni casuali
sample_df = df_analysis.sample(min(9, len(df_analysis)))


plt.figure(figsize=(12, 12))
for i, row in enumerate(sample_df.itertuples()):
    plt.subplot(3, 3, i + 1)
    img = cv2.imread(row.path)

    # Trova i cerchi
    circles = cv2.HoughCircles(cv2.GaussianBlur(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (9, 9), 2), cv2.HOUGH_GRADIENT, 1.2, FINAL_PATCH_SIZE, 50, 30, int(TARGET_DIAMETER / 2 * 0.8), int(TARGET_DIAMETER / 2 * 1.2))

    # --- BLOCCO CORRETTO E ROBUSTO ---
    if circles is not None:
        circles_squeezed = np.squeeze(circles)

        if circles_squeezed.ndim == 1:
            circle = circles_squeezed
        else:
            circle = circles_squeezed[0]

        x, y, r = int(circle[0]), int(circle[1]), int(circle[2])

        cv2.circle(img, (x, y), r, (0, 255, 0), 4) # Cerchio verde
        cv2.circle(img, (x, y), 2, (0, 0, 255), 3) # Punto centrale rosso
    # --- FINE BLOCCO ---

    # Scrivi il diametro sull'immagine
    diameter_text = f"D: {row.measured_diameter}px"
    cv2.putText(img, diameter_text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"Patch: ...{os.path.basename(row.path)[-20:]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

check normalizzato vs non normalizzato gemini


Studio qualità e necessità della normalizzzazione di scala

In [ ]:
import re, os, glob, pandas as pd

RAW_DIR  = "/content/drive/MyDrive/Patches_test/Radio_Patches_Output"
NORM_DIR = "/content/drive/MyDrive/Patches_test/Radio_Patches_Normalized"

def extract_ids(path: str):
    """
    Restituisce (serie, Hidx<int>, hidx<int>)
    compatibile con qualunque stringa tipo:
      • H355_h001_Carbon Textile 5.jpg
      • H487_h001_Scan5.jpg
      • H210_h020_T_0_90_2A.jpg
    """
    name = os.path.splitext(os.path.basename(path))[0]
    m = re.match(r'H(\d{3})_h(\d{3})_(.*)', name)
    if not m:
        return None
    Hidx, hidx, tail = m.groups()
    # serie = prima cifra 1-6 dentro a tail (Scan5 → 5, Carbon Textile 4 → 4, ...)
    m_ser = re.search(r'(\d)', tail)
    if not m_ser:
        return None
    serie = int(m_ser.group(1))
    return serie, int(Hidx), int(hidx)

def build_dict(root: str):
    d = {}
    for p in glob.glob(f"{root}/**/*.jpg", recursive=True):
        key = extract_ids(p)
        if key: d[key] = p
    return d

raw_paths  = build_dict(RAW_DIR)
norm_paths = build_dict(NORM_DIR)
print(f"RAW: {len(raw_paths)}   NORM: {len(norm_paths)}  (chiavi uniche)")


In [ ]:
from collections import defaultdict

# 1) calcola offset Hidx per ogni serie
shift = {}
for s in range(1,7):
    H_raw  = min(k[1] for k in raw_paths  if k[0]==s) or None
    H_norm = min(k[1] for k in norm_paths if k[0]==s) or None
    if H_raw and H_norm:
        shift[s] = H_raw - H_norm        # tip. 0 per CT1-3, -1 per Scan4-6
print("Shift per serie:", shift)

# 2) ricostruisci dizionario NORM con Hidx compensato
norm_shifted = {}
for (s,H,h), path in norm_paths.items():
    H_adj = H + shift.get(s,0)
    norm_shifted[(s,H_adj,h)] = path

# 3) accoppia
pairs = {k: (raw_paths[k], norm_shifted[k])
         for k in raw_paths.keys() & norm_shifted.keys()}

print(f"Patch accoppiate: {len(pairs)} / {len(raw_paths)} RAW  "
      f"({len(raw_paths)-len(pairs)} non trovate)")

# opzionale: salva CSV con mappa
pd.DataFrame(
    [(s,H,h, raw, norm) for (s,H,h),(raw,norm) in pairs.items()],
    columns=["serie","Hidx","hidx","raw_path","norm_path"]
).to_csv("/content/patch_match.csv", index=False)

print("CSV di accoppiamento scritto in /content/patch_match.csv")


In [ ]:
import numpy as np, pandas as pd, cv2, matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

def hough_diam(path, kind):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None: return np.nan
    img = cv2.GaussianBlur(img, (9, 9), 2)
    minR, maxR = ((50, 200) if kind=='raw' else (110, 190))
    cir = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1.2,
                           minDist=img.shape[0], param1=50, param2=30,
                           minRadius=minR, maxRadius=maxR)
    return cir[0,0,2]*2 if cir is not None else np.nan

# ---- misura tutti i diametri (≈6–7 s sui 600 patch) ----
rows = []
for (serie,H,h), (raw_p, norm_p) in pairs.items():
    d_raw  = hough_diam(raw_p,  'raw')
    d_norm = hough_diam(norm_p, 'norm')
    rows.append((serie, H, h, d_raw, d_norm))

df = pd.DataFrame(rows, columns=['serie','H','h','px_raw','px_norm']).dropna()
print(f"{len(df)} coppie con cerchio rilevato")      # dovrebbe restarne >590


In [ ]:
import glob, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

RAW_DIR  = "/content/drive/MyDrive/Patches_test/Radio_Patches_Output"
NORM_DIR = "/content/drive/MyDrive/Patches_test/Radio_Patches_Normalized"
TARGET_PX = 290          # diametro obiettivo dopo normalizzazione

def hough_px(path, is_raw):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None: return np.nan
    img = cv2.GaussianBlur(img, (9,9), 2)
    minR, maxR = ((50,200) if is_raw else (110,190))
    cir = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1.2,
                           minDist=img.shape[0], param1=50, param2=30,
                           minRadius=minR, maxRadius=maxR)
    return cir[0,0,2]*2 if cir is not None else np.nan

# misura diametri
raw_px  = np.array([hough_px(p, True)
                    for p in glob.glob(f"{RAW_DIR}/**/*.jpg", recursive=True)])
norm_px = np.array([hough_px(p, False)
                    for p in glob.glob(f"{NORM_DIR}/**/*.jpg", recursive=True)])

raw_px  = raw_px[~np.isnan(raw_px)]
norm_px = norm_px[~np.isnan(norm_px)]

print(f"diametri letti: RAW={len(raw_px)}   NORM={len(norm_px)}")

# scatter
fig, ax = plt.subplots(1,2, figsize=(12,6))

for i,(data,title,color) in enumerate([
        (raw_px , "PRIMA della normalizzazione", 'steelblue'),
        (norm_px, "DOPO la normalizzazione"   , 'darkgreen')]):

    # ascissa fissa (ma con jitter leggero per separare punti)
    jitter = np.random.uniform(-3, 3, len(data))
    ax[i].scatter(np.full_like(data, TARGET_PX)+jitter,
                  data, alpha=.6, color=color, s=22)

    # retta di riferimento
    ax[i].plot([TARGET_PX-10, TARGET_PX+10],
               [TARGET_PX-10, TARGET_PX+10], 'k--', lw=1.5)

    ax[i].set(title=title,
              xlabel='Diametro nominale (290 px)',
              ylabel='Diametro misurato (px)',
              xlim=(TARGET_PX-15, TARGET_PX+15),
              ylim=(data.min()-20, data.max()+20))

plt.suptitle("Confronto distribuzioni diametro foro – prima / dopo", fontsize=16)
plt.tight_layout(); plt.show()


In [ ]:
#@title istogrammi affiancati a diaetri, mostra effetti normalizzazione su varianza
#come si vede i grupppi di fori sono divisi in due gruppi Raw e dopo diventano uno (verde vs blu)

bins = np.linspace(100, 350, 40)

plt.figure(figsize=(10,4))
plt.hist(raw_px , bins=bins, alpha=.6, label='RAW',  color='tab:blue')
plt.hist(norm_px, bins=bins, alpha=.6, label='NORM', color='tab:green')
plt.axvline(NOMINAL_D_PX, ls='--', color='k', lw=1.5, label='Nominale 290 px')
plt.xlabel('Diametro foro (px)'); plt.ylabel('Conteggio')
plt.title('Istogramma diametri foro'); plt.legend(); plt.show()


In [ ]:
#@title A nuvola diagonale statistica con x variabile per vedere bene #Ordini entrambe le liste: «il più piccolo di prima ↔ il più piccolo di dopo»; «il 2° più piccolo ↔ il 2° più piccolo» … L’effetto visivo è comunque una nuvola diagonale, perché in media i diametri piccoli restano piccoli e i grandi restano grandi – ma Y si addensa attorno a ~290 px.
import numpy as np, matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

# --- assumo che tu abbia già raw_px e norm_px ---
# se no ricalcola i diametri con la tua funzione hough_diam_px

def make_cloud(data, ax, title, jitter=4, color='tab:blue'):
    rng = np.random.default_rng(0)
    N   = len(data)
    x   = np.sort(data) + rng.uniform(-jitter, jitter, N)
    y   = np.sort(data)                     # stesso ordine → diagonale
    ax.scatter(x, y, alpha=.6, s=22, edgecolors='none', color=color)
    lim = y.max()*1.05
    ax.plot([0, lim], [0, lim], 'r--', lw=1.4)
    ax.set(title=title, xlim=(0, lim), ylim=(0, lim),
           xlabel='Diametro (px) – asse X jitterato',
           ylabel='Diametro (px) – asse Y', aspect='equal')

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
make_cloud(raw_px , axes[0], 'RAW (pre-normalizzazione)',  jitter=5, color='tab:blue')
make_cloud(norm_px, axes[1], 'NORM (post-normalizzazione)', jitter=2, color='tab:green')

plt.suptitle('Confronto nuvole diagonali – metodo statistico', fontsize=15)
plt.tight_layout(); plt.show()


In [ ]:
#@title colonne con jitter accentuato
import numpy as np, matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

nominal_px = 290          # riferimento ideale

def jittered_column(data, color, label, jitter=20):
    x = nominal_px + np.random.uniform(-jitter, jitter, len(data))
    plt.scatter(x, data, alpha=.6, s=22, color=color, label=label)

plt.figure(figsize=(6,7))
jittered_column(raw_px , 'tab:blue' , 'PRIMA',  25)
jittered_column(norm_px, 'tab:green', 'DOPO',    6)
plt.axhline(nominal_px, ls='--', color='k', lw=1.2)
plt.xlabel('Diametro nominale (pixel) ± jitter')
plt.ylabel('Diametro misurato (px)')
plt.title('Colonne “larghe” – effetto restringimento')
plt.legend(); plt.show()


graico a violino

In [ ]:
import glob, cv2, numpy as np, pandas as pd
import seaborn as sns, matplotlib.pyplot as plt
sns.set(style='whitegrid')

RAW_DIR  = "/content/drive/MyDrive/Patches_test/Radio_Patches_Output"
NORM_DIR = "/content/drive/MyDrive/Patches_test/Radio_Patches_Normalized"

def hough_px(path, is_raw):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None: return np.nan
    img = cv2.GaussianBlur(img, (9,9), 2)
    minR, maxR = ((50,200) if is_raw else (110,190))
    c = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1.2,
                         minDist=img.shape[0], param1=50, param2=30,
                         minRadius=minR, maxRadius=maxR)
    return c[0,0,2]*2 if c is not None else np.nan

def extract_ids(path):                 # stesso helper di prima
    import re, os
    name = os.path.splitext(os.path.basename(path))[0]
    m = re.match(r'H\d{3}_h\d{3}_(.*)', name)
    if not m: return None
    tail = m.group(1)
    m_ser = re.search(r'(\d)', tail)
    return int(m_ser.group(1)) if m_ser else None

# ---------- misura e costruisci DataFrame unico ----------
rows = []
for folder, label, is_raw in [(RAW_DIR,'RAW',True), (NORM_DIR,'NORM',False)]:
    for p in glob.glob(f"{folder}/**/*.jpg", recursive=True):
        serie = extract_ids(p)
        d_px  = hough_px(p, is_raw)
        if not np.isnan(d_px) and serie:          # scarta subito i NaN
            rows.append((serie, d_px, label))

df_all = pd.DataFrame(rows, columns=['serie','diam_px','stato'])
print(df_all.groupby('stato').size())


In [ ]:
plt.figure(figsize=(11,5))
sns.violinplot(data=df_all, x='serie', y='diam_px',
               hue='stato', split=True, inner='quart',
               palette={'RAW':'#1f77b4', 'NORM':'#2ca02c'})
plt.axhline(290, ls='--', color='k', lw=1.2)
plt.title('Violin-plot diametri per serie (RAW vs NORM)')
plt.xlabel('Serie provino'); plt.ylabel('Diametro (px)')
plt.legend(title=''); plt.show()


In [ ]:
jit_raw, jit_norm = 25, 6
rng = np.random.default_rng(0)
nom_px = 290

fig, ax = plt.subplots(1,2, figsize=(12,6))
for i, (lab, jitter, col) in enumerate([('RAW',jit_raw,'tab:blue'),
                                        ('NORM',jit_norm,'tab:green')]):
    subset = df_all[df_all['stato']==lab]['diam_px'].to_numpy()
    x = nom_px + rng.uniform(-jitter, jitter, len(subset))
    ax[i].scatter(x, subset, alpha=.6, color=col, s=20, edgecolors='none')
    ax[i].set(title=f'{lab} – jitter {jitter}px',
              xlabel='X = nominale ± jitter', ylabel='Diametro (px)')
    ax[i].axhline(nom_px, ls='--', color='k', lw=1.2)
plt.suptitle('Distribuzione diametri prima / dopo con jitter', fontsize=15)
plt.tight_layout(); plt.show()


Fase 1: Accoppiamento dei File (il nostro prerequisito)
Useremo la logica robusta che abbiamo definito prima per creare la nostra lista di coppie di file.

Fase 2: Misurazione dei Diametri
Scriveremo una funzione che prende il percorso di un'immagine, la analizza con OpenCV e restituisce il diametro del foro in pixel. Useremo una tecnica standard e molto affidabile chiamata "Trasformata Circolare di Hough".

Fase 3: Raccolta Dati e Plotting
Applicheremo la funzione di misurazione a ogni file di ogni coppia, raccoglieremo i risultati in una tabella e infine useremo questa tabella per creare il nostro grafico "nuvola diagonale" (che in realtà sarà un grafico di correlazione).

In [ ]:
#CODICE GEMINI solo accoppiamento dei file FASE 1
import os
import glob
from collections import defaultdict
import re
# --- Fase 1: Definire le costanti e le corrispondenze ---

PATCH_RAW_DIR  = '/content/drive/MyDrive/Patches_test/Radio_Patches_Output'
PATCH_NORM_DIR = '/content/drive/MyDrive/Patches_test/Radio_Patches_Normalized'

# Definiamo le corrispondenze tra i nomi delle sottocartelle
# La chiave è il nome nella cartella RAW, il valore è il nome nella cartella NORMALIZED
# NOTA: Adapta questa mappa se i nomi per le prime scan sono diversi
FOLDER_MAP = {
    "Carbon Textile 1A": "Carbon Textile 1A", # Esempio, da verificare
    "Carbon Textile 1B": "Carbon Textile 1B",
    "Carbon Textile 2A": "Carbon Textile 2A",
    "Carbon Textile 2B": "Carbon Textile 2B", # Esempio, da verificare
    "Carbon Textile 3A": "Carbon Textile 3A",
    "Carbon Textile 3B": "Carbon Textile 3B", # Esempio, da verificare
    "Carbon Textile 4": "Scan4",
    "Carbon Textile 5": "Scan5",
    "Carbon Textile 6": "Scan6",
}

# --- Fase 2: Logica di accoppiamento ---

matched_pairs = []
unmatched_raw = []
unmatched_norm = defaultdict(list)

# Iteriamo sulla mappa delle cartelle che vogliamo accoppiare
for raw_folder_name, norm_folder_name in FOLDER_MAP.items():

    raw_subdir = os.path.join(PATCH_RAW_DIR, raw_folder_name)
    norm_subdir = os.path.join(PATCH_NORM_DIR, norm_folder_name)

    print(f"\n--- Processando coppia: '{raw_folder_name}' <--> '{norm_folder_name}' ---")

    # Controlliamo che le cartelle esistano per evitare errori
    if not os.path.isdir(raw_subdir):
        print(f"ATTENZIONE: La cartella RAW non esiste: {raw_subdir}")
        continue
    if not os.path.isdir(norm_subdir):
        print(f"ATTENZIONE: La cartella NORMALIZED non esiste: {norm_subdir}")
        continue

    # Recuperiamo la lista dei file jpg, ORDINATA ALFABETICAMENTE.
    # L'ordinamento è fondamentale.
    raw_files = sorted(glob.glob(os.path.join(raw_subdir, '*.jpg')))
    norm_files = sorted(glob.glob(os.path.join(norm_subdir, '*.jpg')))

    print(f"Trovati {len(raw_files)} file in RAW e {len(norm_files)} in NORMALIZED.")

    # Adesso la parte cruciale. Usiamo un dizionario per rendere il matching robusto
    # La chiave sarà la parte del nome che DEVE essere uguale (es: h001, h002 etc)

    def get_h_key(filepath):
        # Estrae qualcosa tipo "h001" da "H488_h001_Carbon Textile 5.jpg"
        match = re.search(r'_h(\d{3})_', os.path.basename(filepath))
        return match.group(1) if match else None

    raw_dict = {get_h_key(p): p for p in raw_files if get_h_key(p)}
    norm_dict = {get_h_key(p): p for p in norm_files if get_h_key(p)}

    # Troviamo le chiavi in comune
    common_keys = set(raw_dict.keys()) & set(norm_dict.keys())
    print(f"Accoppiati {len(common_keys)} file basati sulla chiave _hXXX_.")

    for key in sorted(list(common_keys)):
        matched_pairs.append((raw_dict[key], norm_dict[key]))

# --- Fase 3: Risultati ---
print("\n==============================================")
print(f"TOTALE PATCH ACCOPPIATE: {len(matched_pairs)}")
print("==============================================")

# Stampiamo i primi 5 accoppiamenti per verifica
print("\nEsempi di accoppiamenti riusciti:")
for i, (raw_p, norm_p) in enumerate(matched_pairs[:5]):
    print(f"Coppia {i+1}:")
    print(f"  RAW:  {os.path.basename(raw_p)}")
    print(f"  NORM: {os.path.basename(norm_p)}")

# Stampiamo l'ultimo accoppiamento trovato per vedere i file problematici
if matched_pairs:
    print("\nUltimo accoppiamento:")
    print(f"  RAW:  {os.path.basename(matched_pairs[-1][0])}")
    print(f"  NORM: {os.path.basename(matched_pairs[-1][1])}")

In [ ]:
#@title Fase 2
# ==============================================================================
# FASE 2: FUNZIONE DI MISURAZIONE DEL DIAMETRO
# ==============================================================================

def measure_hole_diameter(image_path: str) -> float | None:
    """
    Carica un'immagine, trova il cerchio più prominente e restituisce il suo diametro in pixel.
    Restituisce None se non trova nessun cerchio.
    """
    try:
        # Carica l'immagine in scala di grigi
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return None

        # Applica una sfocatura per ridurre il rumore e migliorare il rilevamento
        img_blur = cv2.medianBlur(img, 5)

        # Applica la trasformata di Hough per trovare i cerchi
        # Questi parametri potrebbero richiedere un po' di tuning
        circles = cv2.HoughCircles(
            img_blur,
            cv2.HOUGH_GRADIENT,
            dp=1.2,              # Risoluzione inversa dell'accumulatore
            minDist=400,       # Distanza minima tra i centri dei cerchi rilevati (essendocene solo 1, mettiamo un valore alto)
            param1=60,         # Valore di soglia superiore per l'algoritmo di Canny
            param2=35,         # Soglia per il centro del cerchio
            minRadius=30,      # Raggio minimo del cerchio (per evitare falsi positivi)
            maxRadius=210      # Raggio massimo (il foro normalizzato è ~150px di raggio)
        )

        if circles is not None:
            # Arrotonda le coordinate e il raggio all'intero più vicino
            circles = np.uint16(np.around(circles))
            # Prendiamo solo il primo cerchio trovato (dovrebbe essercene solo uno)
            best_circle = circles[0, 0]
            # Il raggio è la terza componente
            radius = best_circle[2]
            return float(radius * 2) # Ritorna il DIAMETRO

    except Exception as e:
        # print(f"Errore durante l'analisi di {image_path}: {e}")
        return None

    return None

In [ ]:
# ==============================================================================
# OBIETTIVO FINALE: GRAFICO A NUVOLA PRE vs POST NORMALIZZAZIONE
# ==============================================================================
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Prerequisito: Assicuriamoci che la variabile 'matched_pairs' esista e contenga le 596 coppie.
# (Esegui prima la Fase 1 di accoppiamento se non l'hai già fatto)
if 'matched_pairs' not in locals() or len(matched_pairs) < 580:
    raise ValueError("Esegui prima la cella di accoppiamento! 'matched_pairs' non è pronto.")


def measure_hole_diameter_final(image_path: str) -> float | None:
    """
    Funzione di misura OTTIMIZZATA. Sa distinguere tra file raw e normalizzati
    e adatta i parametri di conseguenza.
    """
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None: return None

        img_blur = cv2.medianBlur(img, 5)

        # --- PARAMETRI INTELLIGENTI ---
        # Impostiamo parametri di default
        hough_param2 = 28  # Soglia di confidenza abbassata per essere più permissivi

        # Adattiamo i raggi in base al tipo di file
        is_normalized = "Normalized" in image_path or "Scan" in image_path
        if is_normalized:
            # Per i file normalizzati, i diametri sono molto consistenti
            min_r, max_r = 135, 175 # Range stretto attorno a un diametro di ~300px
        else:
            # Per i file raw, i diametri variano molto
            min_r, max_r = 50, 160  # Range ampio da ~100px a ~320px di diametro
        # --- FINE PARAMETRI INTELLIGENTI ---

        circles = cv2.HoughCircles(
            img_blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=400,
            param1=60, param2=hough_param2,
            minRadius=min_r, maxRadius=max_r
        )

        if circles is not None:
            # Ritorna il diametro del primo (e unico) cerchio trovato
            return float(np.around(circles[0, 0, 2]) * 2)

    except Exception:
        return None
    return None

# --- Misurazione su tutte le coppie ---
results = []
for raw_path, norm_path in tqdm(matched_pairs, desc="Misurazione diametri"):
    raw_diameter = measure_hole_diameter_final(raw_path)
    norm_diameter = measure_hole_diameter_final(norm_path)

    if raw_diameter is not None and norm_diameter is not None:
        results.append({
            'raw_diameter_px': raw_diameter,
            'norm_diameter_px': norm_diameter,
        })

df_results = pd.DataFrame(results)
print(f"\nMisurazione completata. Trovati diametri per {len(df_results)} coppie su {len(matched_pairs)} totali.")

# --- Creazione del GRAFICO FINALE ---
if df_results.empty:
    print("\nERRORE: Nessun diametro è stato misurato. Impossibile creare il grafico.")
    print("Consiglio: riesegui la cella di debug e affina i parametri in 'measure_hole_diameter_final'.")
else:
    print("\nDati pronti per il grafico. Ecco un'anteprima:")
    print(df_results.describe())

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(11, 11))

    # Il grafico a nuvola che volevamo
    sns.scatterplot(
        data=df_results,
        x='raw_diameter_px',      # ASSE X: Diametri originali, disordinati
        y='norm_diameter_px',     # ASSE Y: Diametri normalizzati, consistenti
        alpha=0.6,
        ax=ax
    )

    # Ora la "nuvola" si svilupperà orizzontalmente invece che verticalmente.
    # Evidenziamo la media dei valori normalizzati con una linea ORIZZONTALE
    mean_norm = df_results['norm_diameter_px'].mean()
    ax.axhline(mean_norm, color='red', linestyle='--', linewidth=2, label=f'Media Normalizzata ({mean_norm:.1f} px)')

    ax.set_title('Effetto della Normalizzazione sui Diametri Misurati', fontsize=18, weight='bold')
    ax.set_xlabel('Diametro Foro Originale [pixel]', fontsize=14)
    ax.set_ylabel('Diametro Foro Normalizzato [pixel]', fontsize=14)
    ax.legend(loc='upper left')

    # Forza gli assi ad essere uguali per apprezzare la compressione dei valori
    ax.axis('equal')
    plt.tight_layout()
    plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ==============================================================================
# BLOCCO DI DEBUG INTERATTIVO PER measure_hole_diameter
# ==============================================================================

# 1. Seleziona UNA coppia di immagini di test dalla tua lista di accoppiamenti
# Prendi una coppia a metà lista, che è più rappresentativa
test_raw_path, test_norm_path = matched_pairs[300] # O un altro indice a tua scelta

print(f"File di test RAW: {os.path.basename(test_raw_path)}")
print(f"File di test NORM: {os.path.basename(test_norm_path)}")

def debug_hole_measurement(image_path: str):
    """
    Questa funzione non solo misura, ma mostra anche le immagini intermedie
    per capire perché l'algoritmo fallisce o ha successo.
    """
    print(f"\n--- Analizzando: {os.path.basename(image_path)} ---")

    # Carica l'immagine a colori per poter disegnare in rosso
    img_color = cv2.imread(image_path)
    if img_color is None:
        print("Errore: Impossibile caricare l'immagine.")
        return

    # Converti in scala di grigi per l'elaborazione
    img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

    # Applica una sfocatura per ridurre il rumore
    img_blur = cv2.medianBlur(img_gray, 5)

    # --- INIZIO TUNING PARAMETRI ---
    # Questi sono i "pomelli" su cui dobbiamo lavorare.
    # Prova a cambiare un valore alla volta.
    hough_dp = 1.2
    hough_param1 = 60  # Aumentalo se hai molti bordi spuri (es. 100), diminuiscilo se non trova il bordo principale.
    hough_param2 = 35  # Diminuiscilo se non trova nessun cerchio (es. 25 o 20). È la "confidenza" richiesta.
    hough_min_radius = 40  # Imposta un raggio minimo realistico per le immagini RAW
    hough_max_radius = 200 # Imposta un raggio massimo realistico per le immagini RAW

    # Se stiamo analizzando un'immagine normalizzata, i raggi sono più consistenti
    if "Normalized" in image_path:
        hough_min_radius = 130 # I fori normalizzati sono circa 280-330 px di diametro
        hough_max_radius = 170
    # --- FINE TUNING PARAMETRI ---

    print(f"Parametri Hough: dp={hough_dp}, param1={hough_param1}, param2={hough_param2}, minR={hough_min_radius}, maxR={hough_max_radius}")

    # Applica la trasformata di Hough
    circles = cv2.HoughCircles(
        img_blur,
        cv2.HOUGH_GRADIENT,
        dp=hough_dp,
        minDist=400,
        param1=hough_param1,
        param2=hough_param2,
        minRadius=hough_min_radius,
        maxRadius=hough_max_radius
    )

    # Disegna i risultati per vederli
    if circles is not None:
        print(f"TROVATI {len(circles[0])} CERCHI!")
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Disegna la circonferenza esterna del cerchio
            cv2.circle(img_color, (i[0], i[1]), i[2], (0, 0, 255), 2)  # Rosso
            # Disegna il centro del cerchio
            cv2.circle(img_color, (i[0], i[1]), 2, (0, 255, 0), 3)   # Verde

            diameter = i[2] * 2
            print(f"  - Centro (x,y): ({i[0]},{i[1]}), Diametro: {diameter} px")
    else:
        print("NESSUN CERCHIO TROVATO con questi parametri.")

    # Mostra l'immagine originale e quella con i cerchi trovati
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    axes[0].set_title('Immagine Originale')
    axes[1].imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Risultato Rilevamento Cerchi')
    plt.show()

# --- ESEGUIAMO IL DEBUG ---
debug_hole_measurement(test_raw_path)
debug_hole_measurement(test_norm_path)